# cwwppkgeotaglib.ipynb

(c) 2018, 2019, 2020, 2021  C. W. Wright  
( cwwppkgeotaglib_asof: 2021-1027-2329 )

In [ ]:
#@title Mount your Google Drive as /content/Gdrive  { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  from google.colab import drive
  !rm -rf /content/sample_data/
  drive.mount('/content/.gdrive')
  !ln -sf /content/.gdrive/My\ Drive/  /content/Gdrive
  #!ln -s /content/Gdrive/Missions/NC_Coast/   /content/NC_Coast  
  #!ln -s /content/Gdrive/Missions/Uas/   /content/Uas  
  print('Your Gdrive is mounted')

# Defs ( Library Functions )

In [ ]:
#@title defs, data, cwwppkgeotaglib_asof, etc. { form-width: "25%", display-mode: "form" }

cwwppkgeotaglib_asof = 'cwwppkgeotaglib_asof: 2021-1027-2329'
#
# -*- coding: utf-8 -*-
"""cwwppkgeotaglib.ipynb
 C. W. Wright https://github.com/lidar532/  (C) 2018, 2019, 2020

 Library of functions for generating interpolated photo locations
 from a PPK trajectory file, a file of precision high accuracy photo
 flash events, and a file of photo names with cooresponding EXIF
 time stamps.
"""
asof = cwwppkgeotaglib_asof
global ppk
import sys
import calendar
import datetime
import os
import platform
import re
import urllib
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime

from collections import OrderedDict
from pathlib import Path, PureWindowsPath
import pathlib
from IPython.core.display import HTML, display
from matplotlib import colors
from pandas.plotting import register_matplotlib_converters


try:
  import google.colab
  IN_COLAB = True
  my_platform = 'colab'
except:
  IN_COLAB = False
  my_platform = 'unknown'

# Install pyproj package.
try:
    import pyproj
    print('pyproj module loaded.')
except:
    if 'pyproj' in sys.modules:
        print('Pyproj found. Loaded.')
    else:
        print('Installing pyproj module.')
        !pip install pyproj

from pyproj import Proj

register_matplotlib_converters()

# Changelog
# 	2019-0802 -WW
#		Added ability to estimate the hours and seconds time offset values to fix EXIF data.  It's based on the
#	    last flash and last exif which seem to be more reliable than the first set.
#       
#       Changed load_exif to skip the first picture since it conld be either a header row or a picture.  When it is a header row
#       it causes the time plot to give unusable results. 

# For interactive 
# %matplotlib notebook

In [ ]:
#@title data: ppk_user_settings & ppk { form-width: "25%", display-mode: "form" }
# Settings the user can or must set
ppk_user_settings = { 
    'Photos_to_Skip'                : 0,
    'Accuracy_Scale_Factor'        	: 2.0,
    'Accuracy_Offset_Meters' 		    : 0.05,
	'PixPos_Directory'				        : "",
    'Exif_File_Name'                : "",
    'Trajectory_file_Name'          : "",
	'Trajectory_GPS_to_UTC_Time_difference' : 0,
    'Flash_Events_file_Name'        : "",
    'EXIF_drift_correction_seconds' : 0,
    'EXIF_Offset_from_UTC_Hours'    : 0,
    'Base_station_ID'               : "",
    'User_Notes'                    : "",
    'Generate_Output_File'          : "No",
    'Plot_Times'                    : True,
    'Show_File_Stats'               : True,
    'Show_Flash_event_Distribution' : False,
    'Show_XYZ_Std_Devs'             : False,
    'Show_Photo_Location_Plan_View' : False,
    'Show_Photo_Elevations'         : False,
    'Output_Start_Time'             : "2020-0101 00:00:00",
    'Output_End_Time'               : "2020-0101 23:59:59",
    'Debugging_Output'              : "None",
    'User_marker'                   :  0.0,
    'Method'                        : 'None'
}

ppk = { 'ppk_flash_sync_version'      : f'cwwppkgeotaglib.py asof {asof}',   # Configure ppk variables
		'Notebook Program'		            : 'Not Set',
        'run_error'                   : '',
        'run'                         : True,
        'debug'                       : 0,
        'Zulu_offset_hours_exif'      : 4.0,
        'exif_offset_seconds'         : -8.0,
        'GPS_base_station_name'       : '',
        'run_time_date'               : '',
        'ofn'                         : '',
        'traj_root_name'              : '',
        'npix'                        : 0,
        'nflash'                      : 0,
        'output_records'              : 0,
        'output_record_sep'           : ',',
        'gps_leap_seconds'            : 0,
        'sec_offset_str'              : 's0',
        'hours_offset_str'            : 'h0',
        'flash_stamp_duplicate_count' : 0,
        'exif_df'                     : '',
        'Missing_Flashs'              : 0,
        'exif_duptimes_count'         : 0,
        'Trajectory_Generator'        : 'none'
      }



In [ ]:
#@title defs: gen_output_file_name( ppk, ppk_user_settings) { form-width: "25%", display-mode: "form" }
def gen_output_file_name( ppk, ppk_user_settings, tail='-pixpos.txt'):
  """Computes an output filename based on the trajectory file plus the EXIF
  clock offsets in hours and seconds.  The computed filename is returned and
  and placed in ppk['ofn'].

  Parameters:
  -----------
    ppk
    ppk_user_settings
    tail = '-pixpos.txt'

  Returns:
  --------
    A full path fiilename.
    
  """
  start_end_times = ppk_user_settings['Output_Start_Time'].replace(':','').split()[1] + '-' + ppk_user_settings['Output_End_Time'].replace(':','').split()[1]
  currentDT = ppk['run_time_date']
  ppk['sec_offset_str']    = 's'+str(ppk_user_settings['EXIF_drift_correction_seconds'])
  ppk['hours_offset_str']  = 'h'+str(ppk_user_settings['EXIF_Offset_from_UTC_Hours'])
  ppk['traj_root_name'] = os.path.splitext(os.path.basename(ppk_user_settings['Trajectory_file_Name'] ))[0]
  ofn = ppk_user_settings['PixPos_Directory'] +'/'+\
    ppk['traj_root_name']+'-'+start_end_times+\
    '-'+'h'+str(ppk_user_settings['EXIF_Offset_from_UTC_Hours'])+\
    's'+str(ppk_user_settings['EXIF_drift_correction_seconds'])+tail
  ppk['ofn'] = ofn
  return ofn
  


In [ ]:
#@title defs: check_input_files( ppk, ppk_user_settings ): { form-width: "25%", display-mode: "form" }
def check_input_files( ppk, ppk_user_settings ):
  """Checks the input files found in ppp_user_settings[] for existence.  
  If they do not exist, it sets ppk['run'] to False.
  """
  if ppk['debug'] > 0:  print('check_input_files()')
  currentDT = ppk['run_time_date'] = datetime.datetime.now()  # Get current time/date to mark running this script today
  ppk['run'] = True
  ppk['ofn'] = "Not Set"
  if ppk['debug'] > 0:
    print("\n    Base Station: ", ppk['GPS_base_station_name'],
          "\n", os.path.isfile( ppk_user_settings['Exif_File_Name'] ), " Exif File:", ppk_user_settings['Exif_File_Name'],
          "\n", os.path.isfile( ppk_user_settings['Trajectory_file_Name'] ),   "  PPK File:", ppk_user_settings['Trajectory_file_Name'], 
          "\n", os.path.isfile( ppk_user_settings['Flash_Events_file_Name']), " Flash File:",  ppk_user_settings['Flash_Events_file_Name'],
          '\n Output Filename:', ppk['ofn'], )
    
  if os.path.isfile( ppk_user_settings['Exif_File_Name'] ) == False:
      ppk['run']  = False
      ppk['run_error'] = f"Exif File not found: {ppk_user_settings['Exif_File_Name']}"
      return ""
  if os.path.isfile( ppk_user_settings['Trajectory_file_Name'] )   == False:
      ppk['run'] = False 
      ppk['run_error'] = f"Trajectory File not found: {ppk_user_settings['Trajectory_file_Name']}"
      return ""
  if os.path.isfile( ppk_user_settings['Flash_Events_file_Name'] ) == False:
      ppk['run'] = False
      ppk['run_error'] = f"Flash Events File not found: {ppk_user_settings['Flash_Events_file_Name']}"
      return ""  
  if ppk['debug'] > 0:        
      print('\n', ppk_user_settings['EXIF_drift_correction_seconds'],   " Camera clock drift correction\n",
            ppk_user_settings['EXIF_Offset_from_UTC_Hours']," Camera offset to UTC\n")
  return ppk['ofn']


# Test code for the cell above goes here.
if __name__ == '__main__':
  ;



In [ ]:
#@title def format_single_time(x): { form-width: "25%", display-mode: "form" }
def format_single_time(x):
  # s = print(x.strftime("%b %d %Y %H:%M:%S\t"), end="")     
  return x.strftime("%b %d %Y %H:%M:%S")



In [ ]:
#@title def show_times( exif_df, ppk_trj, flashs): { form-width: "25%", display-mode: "form" }
def show_times( exif_df, ppk_trj, flashs):
  first_photo_time = exif_df['Correct_exif_Ztime'].min()
  last_photo_time  = exif_df['Correct_exif_Ztime'].max()
  first_trj_time = ppk_trj['Ztime'].min()
  last_trj_time  = ppk_trj['Ztime'].max()
  first_flash_time = flashs['Flash_Ztime'].min()
  last_flash_time = flashs['Flash_Ztime'].max()

  print( "     Corrected Photo Exif times")
  print( "     Start Time                  Stop Time")
  print( "     ", format_single_time(first_photo_time),  format_single_time(last_photo_time),   " Photo EXIF times" )
  print( "     ", format_single_time(first_flash_time),  format_single_time(last_flash_time),   " Flash event Times")
  print( "     ", format_single_time(first_trj_time),    format_single_time(last_trj_time),     " Trajectory Times" )
  return



In [ ]:
#@title def load_exif(ppk, ppk_user_settings, duplicates='remove', show_duplicates=False ). { form-width: "25%", display-mode: "form" }
####   See encoding option to fix reading from exiftool written from window 10 > file.    
def load_exif( ppk, ppk_user_settings, duplicates='remove', show_duplicates=False ):
  """Load the EXIF datafile to a pandas dataframe ( exif_df )
       1) Load in the exif information from a file
       2) Convert the date/time string to actual time and add as a column to the data frame
       3) Adjusts EXIF time by user supplied corections in ppk_user_settings['EXIF_drift_correction_seconds']
          and ppk_user_settings['EXIF_Offset_from_UTC_Hours']
     ppk_user_settings['EXIF_Offset_from_UTC_Hours']= pd.to_timedelta( ppk_user_settings['EXIF_Offset_from_UTC_Hours'], unit="h")

     Input: ppk, 
            ppk_user_settings                   Uses ['Exif_File_Name'], ['Photos_to_Skip'],['EXIF_Offset_from_UTC_Hours']
                                                ['EXIF_drift_correction_seconds'],
            duplicates={'remove', ''}           Doesn't do anything yet.
            show_duplicates = {False, True}     Default = False.  Printout for user to see photos with duplicate time stamps


     Outputs: Sets ppk['run'], ppk['run_error']


     Returns: exif_df pandas dataframe.

     Sets Globals: exif_duptimes
  """
  global exif_duptimes
  if ppk['debug'] > 0:
      print('load_exif() ')

  exif_df = pd.read_csv( ppk_user_settings['Exif_File_Name'], 
                        names=['pix', 'raw_exif_ymd_hms', 'iso', 'shutter' ],
                        delimiter=',',
                        comment='#',
                        skiprows = 0 )
  

  exif_df = exif_df[ppk_user_settings['Photos_to_Skip']:]
  # Convert date/time to actual time value and add as a column time
  exif_df['exif_time'] = pd.to_datetime(exif_df['raw_exif_ymd_hms'], format=" %Y:%m:%d %H:%M:%S", errors='coerce')
  exif_df['exif_time'] = pd.to_datetime(exif_df['raw_exif_ymd_hms'], format="%Y:%m:%d %H:%M:%S", errors='coerce')

  # Generate a correct exif_Ztime from the EXIF time stamp corrected for drift and UTC offset
  exif_df['Correct_exif_Ztime'] = exif_df['exif_time'] + \
                                  pd.to_timedelta( ppk_user_settings['EXIF_Offset_from_UTC_Hours'], unit="h")+ \
                                  pd.to_timedelta( ppk_user_settings['EXIF_drift_correction_seconds'], unit="s")
  exif_df['Correct_exif_ZtimeIdx'] = exif_df['Correct_exif_Ztime']
  exif_df = exif_df.set_index('Correct_exif_ZtimeIdx')
  ppk['npix']   = exif_df.shape[0]
  if pd.isnull(exif_df['exif_time'][1]):
    ppk['run'] = False
    ppk['run_error'] = f"exif_df is invalid. The EXIF File format is wrong: {ppk_user_settings['Exif_File_Name']}"

  exif_duptimes  = exif_df[exif_df.duplicated( subset='raw_exif_ymd_hms', keep='first')]
  exif_duptimes_count = exif_duptimes.count()['pix']
  ppk['exif_duptimes_count'] = exif_duptimes_count;
  print(f'{exif_duptimes_count} photos with duplicate time stamps found.')

  # save out duplicates to a file for user review.
  if ppk['exif_duptimes_count']:
    exif_duplicates_fn = ppk_user_settings['Exif_File_Name'].replace('.txt', '_duplicates.txt')
    with open(exif_duplicates_fn, 'w' ) as fs_odf:
      fs_odf.write( exif_duptimes.to_string())
    print(f'{exif_duptimes_count*2} photos with the same EXIF time stamp found.')
    if show_duplicates:
      display( exif_duptimes )

  return exif_df

junk='''
#@title Test load_exif(). {form-width: "25%"}
# The following used to test with 2020-0802 NC dataset.
if __name__ == '__main__' and IN_COLAB:
  print(f'photos to skip: {ppk_user_settings["Photos_to_Skip"]}')
  ###exif_fn = "/content/NC_Coast/2020-0802-NC-gps/01_gps/aircraft/exif/2020-0802-v2-F1-exif.txt" #@param {type:"string"}
  ppk_user_settings['Exif_File_Name'] = exif_fn
  exif_df = load_exif( ppk, ppk_user_settings, duplicates='remove', show_duplicates=False )

#@title Testing:. Display exif_duplicates{form-width: "25%"}
if __name__ == '__main__' and IN_COLAB:
  display(exif_duptimes)
'''


In [ ]:
#@title def load_ppkca_trj( ppk, ppk_user_settings ): { form-width: "25%", display-mode: "form" }
def load_ppkca_trj( ppk, ppk_user_settings ):
	fn = pathlib.Path(ppk_user_settings['Trajectory_file_Name'] )
	if fn.exists() == False:
		print(fn," does not exist.")
		ppk['run'] = False
	else:
		try:
			gps_to_utc = ppk_user_settings['Trajectory_GPS_to_UTC_Time_difference']
			print('gps_to_utc:', gps_to_utc )
			ppp_pos = pd.read_csv(ppk_user_settings['Trajectory_file_Name'], skiprows=5, sep='\s+' )
			ppk_data = ppp_pos
			ppk_data['Ztime']       = pd.to_datetime( ppp_pos['YEAR-MM-DD'] +" "+ppp_pos['HR:MN:SS.SS'] ) \
										+ pd.to_timedelta(gps_to_utc, unit='seconds')
			ppk_data['GPS_lat']     =  ppp_pos['LATDD']  + ppp_pos['LATMN']/60.0 + ppp_pos['LATSS']/3600.0
			ppk_data['GPS_lon']     = -( abs(ppp_pos['LONDD']) + ppp_pos['LONMN']/60.0 + ppp_pos['LONSS']/3600.0 )
			ppk_data['GPS_nad83h']  = ppp_pos['HGT(m)']
			ppk_data['sdu']         = ppp_pos['SDHGT(95%)'] / 1.95
			ppk_data['ns']          = ppp_pos['NSV']
			ppk_data['Q']           = 6
			ppk_data['Ztime_idx'] = ppk_data['Ztime']
			ppk_data = ppk_data.set_index('Ztime_idx')
			return ppk_data
		except:
			print(f"{ppk_user_settings['Trajectory_file_Name']} does not appear to be a PPP-ca file and pd.read_csv failed." )
			ppk['run'] = False
			ppk['run_error'] = f"{ppk_user_settings['Trajectory_file_Name']} does not appear to be a PPP-ca file and pd.read_csv failed."
			return {}





In [ ]:
#@title def: load_rtklib_trj( ppk, ppk_user_settings ) { form-width: "25%", display-mode: "form" }
def load_rtklib_trj( ppk, ppk_user_settings ):
    """
    Loads a .pos "Position file" generated by RTKLIB into the pandas dataframe ppk_trj.  The .pos file
    is expected to contain UTC time and not GPS time.
    """
    
    if ppk['debug'] > 0: print('load_rtklib_trj()')
    ppk_trj = pd.read_csv( ppk_user_settings['Trajectory_file_Name'], 
                          names=['date', 'zhms', 
                                 'GPS_lat', 'GPS_lon', 'GPS_nad83h','Q', 'ns',
                                 'sdn', 'sde', 'sdu', 'sdne', 'sdeu', 'sdun', 'age', 'ratio'],
                          delim_whitespace=True,
                          comment='%',
                          skiprows = 1 )
    # ppk_trj
    ppk_trj['Ztime'] = pd.to_datetime( ppk_trj['date']+' '+ppk_trj['zhms'])    # +pd.Timedelta( ppk_trj['date'])
    ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
    ppk_trj = ppk_trj.set_index('Ztime_idx')

    if ppk['debug'] > 0:
        # ppk_trj[0:4][['date','zhms','GPS_lat','GPS_lon','GPS_nad83h','Q','ns']]    
        display(ppk_trj[1:5])
    return ppk_trj

#if __name__ == '__main__' and IN_COLAB:
#  ppk_user_settings['Trajectory_file_Name'] = '/content/NC_Coast/2020-0808-NC-gps/01_gps/trajectories/2020-0808-173643-N7251F-GP173612-NCDU-RK12N-cmb-pos.txt'
#  display(load_rtklib_trj( ppk, ppk_user_settings )[0:2])

In [ ]:
#@title load_grafnav_trj( ppk, ppk_user_settings  ): { form-width: "25%", display-mode: "form" }
def load_grafnav_trj( ppk, ppk_user_settings ):
  data = pd.DataFrame()
  ppk_trj = pd.read_csv(ppk_user_settings['Trajectory_file_Name'],
                    header=None, 
                    skiprows=50,
                    index_col=False, 
                    infer_datetime_format=True,
                    parse_dates=[['date','utc']],
                    delim_whitespace=True,
                    names=['UTMeasting', 'UTMnorthing', 'navd88', 'GPS_lat', 'GPS_lon', 'GPS_nad83h', 'Q', 'sdu', 'sdne', 'date', 'utc' ],
                    skipinitialspace=True )

  ppk_trj['Ztime']     = ppk_trj['date_utc']
  ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
  ppk_trj              = ppk_trj.set_index('Ztime_idx')
  ppk_trj['ns']        = ppk_trj['age'] = 0
  ppk_trj['ratio']     = ppk_trj['sde'] = ppk_trj['sdn'] = ppk_trj['sdeu'] = ppk_trj['sdun']= 0.0
  ppk_trj['date']      = ppk_trj['zhms'] = 0

  # ['age' 'ratio' 'sde' 'sdn' 'sdeu' 'sdun']
  #data['hms_z'] = ppk_trj['Ztime']
  #data['GPS_lat']   = ppk_trj['GPS_lat']
  #data['GPS_lon']   = ppk_trj['GPS_lon']
  #data['GPS_elev'] = ppk_trj['GPS_nad83h']
  #data['q']    = ppk_trj['Q']
  #data['sdu95'] = ppk_trj['sdu']*1.95
  #data['sdu']   = ppk_trj['sdu']
  #data['ns']    = 0
  return ppk_trj

junk = '''
#@title Test Load grafnav trajectory
if __name__ == '__main__' and IN_COLAB:
  ###grafnav_fn = "" #@param {type:"string"}  
  grafnav_df = load_grafnav_trj( ppk, ppk_user_settings )
  display(grafnav_df)
'''


In [ ]:
#@title def determine_trajectory_generator(fn): { form-width: "25%", display-mode: "form" }
def determine_trajectory_generator(fn):
  """
  Scan the first 50 lines of a file searching for grafnav, or rtklib.  
  Input: trajectory file
  Returns: 'none' | 'grafnav' | 'rtklib' | 'ppp-ca'

   """
  max_lines = 50
  rv = 'none'
  with open( fn,  'rt') as fd:
    for line in fd:
      if line.lower().find('grafnav') >= 0 :
        rv = 'grafnav'
        break
      elif line.lower().find('rtklib') >= 0 :
        rv = 'rtklib'
        break
      elif line.lower().find('rtkpost') >= 0:
        rv = 'rtklib'
        break;
      elif line.upper().find('HDR ADR GOVERNMENT OF CANADA') >=  0:
        rv = 'ppp-ca'
        break;
      max_lines = max_lines - 1
      if max_lines == 0:
        break
  return rv

junk='''
#@title Test: determine_trajectory_generator(fn): {form-width: "25%"}
if __name__ == '__main__' :
  #fn = "/content/NC_Coast/2020-0928-NC_GNSS/aircraft/trajectories/2020-0928-153131-N7251F-GP153112-NCDU-RK12N-cmb-conf.txt" #@param {type:"string"}
  print(f' found: {determine_trajectory_generator(fn) } ')
# '''

In [ ]:
#@title def load_k706_flash_events( ppk, ppk_user_settings, duplicates='remove', save_flash_to_file=True ): { form-width: "25%", display-mode: "form" }
def load_k706_flash_events( ppk, ppk_user_settings, duplicates='remove', save_flash_to_file=False ):
  """
  Load a ComNav K706 or K501 GNSS file containing #MARKTIMEA event records. These contain the 
  flash shoe "Event" time stamp data marks captured when the camera shutter fires.

  Inputs:
    ppk, 
    ppk_user_settings, 
    duplicates=['remove', 'return', 'keep']     # Default is to remove the duplicates.

  Returns:
      flash_stamps_df Pandas data frame.

  """
  if ppk['debug'] > 0:
      print('load_k706_flash_events()')
  cols = ['gps_week', 'sow', 't1', 't2', 'Zulu Offset', 'cksum' ]
  flash_stamps_list = []
  gga_list = []
  flash_stamps_f = open( ppk_user_settings['Flash_Events_file_Name'] )
  rx = re.compile(r'#MARKTIMEA,*')
  #rxgga = re.compile(r'\$GPGGA,*')
  for line in flash_stamps_f:
      w = rx.findall(line)
  #    wgga = rxgga.findall(line)
      if w :
          s = line.split(";")
          s = s[1].split(',')
          flash_stamps_list.append( s )
  #    if wgga :
  #        s = line.split(',')
  #        gga_list.append( s )

  # if the flash_stamps_list is empty, then no records found.  File is invalid.
  if flash_stamps_list == []:
    ppk['run'] = False
    print(f'load_k706_flash_events(): No valid data found in { ppk_user_settings["Flash_Events_file_Name"] }')
    return -1

  gga_df = pd.DataFrame.from_records(gga_list)
  # , parse_dates=True, date_parser=nmea2datetime

  ##  unit of the arg (D,h,m,s,ms,us,ns) denote the unit, which is an integer/float number
  flash_stamps_df = pd.DataFrame.from_records( flash_stamps_list, columns=cols )
  ppk['gps_leap_seconds'] = flash_stamps_df['Zulu Offset'].min()
  flash_stamps_df['gps_week'   ] = pd.to_numeric(flash_stamps_df['gps_week'])

  # Using the GPS week value, compute the days offset to pandas time.        
  gps_week_offset = pd.to_timedelta( (flash_stamps_df['gps_week'][1]+52*10)*7+17, unit="D")

  flash_stamps_df['gps_day'    ] = pd.to_timedelta(flash_stamps_df['gps_week']*7, unit = 'D' )
  flash_stamps_df['Zulu Offset'] = pd.to_numeric(flash_stamps_df['Zulu Offset'])
  flash_stamps_df['sow'        ] = pd.to_numeric(flash_stamps_df['sow'])
  flash_stamps_df['gps_sow'    ] = pd.to_timedelta(flash_stamps_df['sow'], unit='s')
  flash_stamps_df['Flash_Ztime'] = pd.to_datetime(flash_stamps_df['sow']+ flash_stamps_df['Zulu Offset'], unit='s')
  flash_stamps_df['Flash_Ztime'] = flash_stamps_df['Flash_Ztime'] + gps_week_offset
  flash_stamps_df['Flash_ZtimeIdx'   ] = flash_stamps_df['Flash_Ztime']
  flash_stamps_df['t1'         ] = pd.to_numeric(flash_stamps_df['t1'])
  flash_stamps_df['t2'         ] = pd.to_numeric(flash_stamps_df['t2'])
  flash_stamps_df['Seconds_Offset'] = int(np.timedelta64( ppk_user_settings['EXIF_drift_correction_seconds'] ).astype(int) / 1e6)
  

  ppk['nflash'] = flash_stamps_df.shape[0]
  flash_stamps_df = flash_stamps_df.set_index('Flash_ZtimeIdx')
  if ppk['debug'] > 0:
      print("GPS Leap Seconds: ", ppk['gps_leap_seconds'], "\n")
      print(ppk['nflash'], " Flash Time Stamps found in File: ", ppk_user_settings['Flash_Events_file_Name'] )
      print(  ppk['npix'], "       Photos found in EXIF file: ", ppk_user_settings['Exif_File_Name'] )
  if ppk['debug'] > 1:
      display( flash_stamps_df[0:10]  )

  duplicate_flash_stamps_df = flash_stamps_df[flash_stamps_df.duplicated( subset='Flash_Ztime', keep='first')]

  # If any duplicates, write them out to a file.
  duplicate_count = duplicate_flash_stamps_df['Flash_Ztime'].count()
  ppk['flash_stamp_duplicate_count'] = duplicate_count
  if duplicate_count:
    flash_stamps_duplicates_fn = ppk_user_settings['Flash_Events_file_Name'].replace('.TXT', '-flash_event_duplicates.txt')
    with open(flash_stamps_duplicates_fn, 'w' ) as fs_odf:
      fs_odf.write( duplicate_flash_stamps_df.to_string())
    print(f'Duplicate Flash Events Found:{duplicate_count}')
    #display( duplicate_flash_stamps_df )

  if duplicates == 'remove':      # keep only the unique records
    flash_stamps_df.drop_duplicates(subset='Flash_Ztime', keep='first', inplace=True)
  elif duplicates == 'return':    # keep only the duplicate records
    flash_stamps_df = duplicate_flash_stamps_df
  
  # Save a human readable version of flash_stamps_df to a file.
  # Fix this to not overwrite.
  if save_flash_to_file:
    flash_stamps_fn = ppk_user_settings['Flash_Events_file_Name'].replace('.TXT', '-flash_events.txt')
    with open(flash_stamps_fn, 'w' ) as fs_odf:
      fs_odf.write( flash_stamps_df.to_string())
  return flash_stamps_df

junk='''
#@title Test flash_events_file {form-width: "35%"}
# The following used to test with 2020-0802 NC dataset.
if __name__ == '__main__' and IN_COLAB:
  ###Flash_Events_file_Name = "" #@param {type:"string"}
  ppk_user_settings['Flash_Events_file_Name'] = Flash_Events_file_Name
  flash_stamps_df = load_k706_flash_events( ppk, ppk_user_settings, duplicates='remove' )
'''




In [ ]:
#@title def interpolate_flash_positions_2( ppk, ppk_user_settings, ppk_trj, flash_stamps_df). { form-width: "25%", display-mode: "form" }
def interpolate_flash_positions_2( ppk, ppk_user_settings, ppk_trj, flash_stamps_df):
  """
  Interpolate the flash event positions using a ppk trajectory from RTKlib or Grafnav.

  Inputs:
          ppk
          ppk_user_settings
          ppk_trj
          flash_stamps_df

  Returns:
          flash_stamps_df

  """
  debug = False
  if debug:
    print('enter interpolate_flash_positions_2()')
  epoch_start = datetime.datetime(1970, 1, 1)
  x0 = (flash_stamps_df['Flash_Ztime'] - epoch_start) / np.timedelta64(1, 'us')
  x1 = (ppk_trj['Ztime'] - epoch_start ) / np.timedelta64(1, 'us')
  ppk_trj.index = ppk_trj.index.round('10us')

  flash_stamps_df.insert(0,'PIX_lat',    np.interp( x0, x1, ppk_trj['GPS_lat'   ]) )
  flash_stamps_df.insert(1,'PIX_lon',    np.interp( x0, x1, ppk_trj['GPS_lon'   ]) )
  flash_stamps_df.insert(2,'PIX_nad83h', np.interp( x0, x1, ppk_trj['GPS_nad83h'   ]) )
  #flash_stamps_df['PIX_lat'   ] = np.interp( x0, x1, ppk_trj['GPS_lat'   ])
  #flash_stamps_df['PIX_lon'   ] = np.interp( x0, x1, ppk_trj['GPS_lon'   ])
  #flash_stamps_df['PIX_nad83h'] = np.interp( x0, x1, ppk_trj['GPS_nad83h'])

  cols_to_drop = [
                  'cksum', 
                  'gps_day', 't1', 't2', 'age', 'ratio',
                  'sdne', 'sde', 'sdn', 'sdeu', 'sdun',
                  'Seconds_Offset', 'gps_week', 'Ztime',
                  'GPS_lat', 'GPS_lon', 'GPS_nad83h'
                  ]
  
  flash_stamps_df.index = flash_stamps_df.index.round('1s')
  flash_stamps_df = flash_stamps_df.join( ppk_trj, how='left')      # how='inner'
  flash_stamps_df = flash_stamps_df.drop(columns=cols_to_drop)
  
  flash_stamps_df['Accuracy_Z'] = flash_stamps_df['sdu']*ppk_user_settings['Accuracy_Scale_Factor']+ppk_user_settings['Accuracy_Offset_Meters']
  flash_stamps_df = flash_stamps_df[['PIX_lat', 'PIX_lon', 'PIX_nad83h', 'Accuracy_Z', 'sdu','Q', 'ns',  'Flash_Ztime', 'gps_sow', 'date', 'zhms', 'sow', 'Zulu Offset']]
  #display(flash_stamps_df)
  if debug:
    print('exit interpolate_flash_positions_2()')
  return flash_stamps_df



In [ ]:
#@title def interpolate_flash_positions(ppk, ppk_user_settings, exif_df, ppk_trj, flash_stamps_df)  ( Old legacy code.) { form-width: "25%", display-mode: "form" }
def interpolate_flash_positions(ppk, ppk_user_settings, exif_df, ppk_trj, flash_stamps_df):
  """
  Interpolate the lat/lon/elevation and altitude  for each record in flash_stamps_df and join other data from the ppk file.
  Interpolate the lat, Lon, and Altitude from the trajectory to the flash time stamps

  Inputs: ppk, ppk_user_settings, exif_df, ppk_trj, flash_stamps_df

  Outputs:
  """
  #trig_lat = np.interp( flash_stamps_df['Ztime'], ppk_trj['Ztime'], ppk_trj['lat'])

  if ppk['debug'] > 0:
      print('interpolate_flash_positions()')

  # Save a copy of flash_stamps_df to Y so we can restore it later and also make a copy to
  # Xflash_stamps_df.
  Y = flash_stamps_df
  Xflash_stamps_df = flash_stamps_df

  # Round to the nearest second on the index column
  Xflash_stamps_df.index = flash_stamps_df.index.round('1s')
  Xflash_stamps_df       = flash_stamps_df.join( ppk_trj, how='inner')

  ee = datetime.datetime(1970, 1, 1)
  x0 =  flash_stamps_df['Flash_Ztime'] - ee
  x1 =  ppk_trj['Ztime'] - ee
  y0 =  ppk_trj['GPS_lat']

  xx0 = x0 / np.timedelta64(1, 'us')
  xx1 = x1 / np.timedelta64(1, 'us')

  ppk_trj.index = ppk_trj.index.round('10us')

  # Interpolate lat, long, and elevation from the trajectory into the flash data frame
  flash_stamps_df['PIX_lat'   ] = np.interp( xx0, xx1, ppk_trj['GPS_lat'   ])
  flash_stamps_df['PIX_lon'   ] = np.interp( xx0, xx1, ppk_trj['GPS_lon'   ])
  flash_stamps_df['PIX_nad83h'] = np.interp( xx0, xx1, ppk_trj['GPS_nad83h'])

  # Merge in the other columns from the ppk_trj to pickup the stats, number of sats, and such
  flash_stamps_df = flash_stamps_df.join( ppk_trj, how='inner')

  # display(flash_stamps_df[30:34][['Flash_Ztime', 'PIX_lat', 'PIX_lon', 'PIX_nad83h','GPS_lat','GPS_lon','GPS_nad83h', 'Q', 'ns', 'sdu', 'age', 'ratio']] )
  Xflash_stamps_df = flash_stamps_df
  flash_stamps_df = Y

  # Add in the camera data, iso, shutter time, etc. from the exif dataframe into te output record.
  output_data = exif_df.join( Xflash_stamps_df, how='inner' )
  ppk['output_records'] = output_data['pix'].count()
  output_data['Seconds_Offset'] = ppk_user_settings['EXIF_drift_correction_seconds']
  output_data['Accuracy_Z'] = output_data['sdu']*ppk_user_settings['Accuracy_Scale_Factor']+ppk_user_settings['Accuracy_Offset_Meters']
  print('Output_data records found:', output_data['pix'].count())
  return output_data




In [ ]:
#@title def sync_with_exif(ppk, ppk_user_settings, exif_df, flash_stamps_df) { form-width: "25%", display-mode: "form" }
def sync_with_exif(ppk, ppk_user_settings, exif_df, flash_stamps_df):
  """
 Sync data based on the the interpolated flash events and the camera EXIF data.

  Inputs: ppk, ppk_user_settings, exif_df, flash_stamps_df

  Outputs:
  """

  if ppk['debug'] > 0:
      print('sync_with_exif()')

  # Add in the camera data, iso, shutter time, etc. from the exif dataframe into te output record.
  output_data = exif_df.join( flash_stamps_df, how='inner' )
  ppk['output_records'] = output_data['pix'].count()
  output_data['Seconds_Offset'] = ppk_user_settings['EXIF_drift_correction_seconds']
  output_data['Accuracy_Z'] = output_data['sdu']*ppk_user_settings['Accuracy_Scale_Factor']+ppk_user_settings['Accuracy_Offset_Meters']
  return output_data


#@title def Generate_output_2(odfn, output_df, Generate_output='Yes')
def Generate_output_2(odfn, output_df, Generate_output='Yes'):
  header = generate_output_header_2(ppk, ppk_user_settings )
  print(header)
  if Generate_output == 'Yes':
    odf = open( odfn , 'w')
    # print(f'odfn:{odfn}')
    print(header, file=odf)
    '''
    output_df.to_csv( odf,
                    sep= '\t', index=False,
                    columns = [ 'pix','PIX_lat','PIX_lon','PIX_nad83h', 'Accuracy_Z', 'Flash_Ztime', 'Q', 'ns', 'sdu', 'gps_sow' ],
                    header = [
                        'Photos                              ', 
                        'PIX_Lat      ',
                        'PIX_Lon    ',
                        'PIX_Elev    ',
                        'Accuracy_Z   ',
                        'Flash_Event_UTC_DATE_Time',
                        '  Q',
                        'N',
                        ' ZStdev    ',
                        'GPS_Time'
                    ],
                    float_format='%10.8f',
                  index_label = '# Date     UTC Time'
                  )
    '''
    out_list = [ 'pix','PIX_lat','PIX_lon','PIX_nad83h', 'Accuracy_Z', 'Flash_Ztime', 'Q', 'ns', 'sdu', 'gps_sow' ]
    print( output_df[out_list].to_string( formatters= {'pix':'{:s}'.format,
                                                      'PIX_lat':'{:12.9f}'.format,
                                                      'PIX_lon':'{:12.9f}'.format,
                                                      'PIX_nad83h':'{:7.3f}'.format,
                                                      'Accuracy':'{:4.2f}'.format,
                                                      'Q':'{:1.0f}'.format,
                                                      'ns':'{:2.0f}'.format,
                                                      'gps_sow':'{:}'.format
                                                      }, index=False), file=odf )
    # 'gps_sow':lambda x:'{:%H%M%S}'.format(pd.to_datetime(x,unit='s'))
    odf.close()   #'gps_sow':'{:}'.format
  else:
    print('No output file generated.')



In [ ]:
#@title def generate_output_header_2(ppk, ppk_user_settings)  { form-width: "25%", display-mode: "form" }
def generate_output_header_2(ppk, ppk_user_settings ):
  currentDT = datetime.datetime.now()
  hdr = f'#1 C. W. Wright https://github.com/lidar532  This file generated: {str(currentDT)} by: {ppk["Notebook Program"]} using: {ppk["ppk_flash_sync_version"]}\n' + \
        f'#2   Trajectory Gen: {ppk["Trajectory_Generator"]}       User Notes: {ppk_user_settings["User_Notes"]} \n' + \
        f'#3  Trajectory File: {ppk_user_settings["Trajectory_file_Name"]}\n' + \
        f'#4 Flash Event File: {ppk_user_settings["Flash_Events_file_Name"]} \n' + \
        f'#5        EXIF File: {ppk_user_settings["Exif_File_Name"]} \n' + \
        f'#6             EXIF Correction   Flash   Missing Photo Duplicate             Accuracy   Sync       Start Time           Stop Time\n' + \
        f'#7 GPS Base ID Hours   Seconds   Events  Events  Count EXIF Time  Marker  Scale Offset  Method      Marker               Marker\n' + \
        f'#8 {ppk_user_settings["Base_station_ID"].rjust(11)} {ppk_user_settings["EXIF_Offset_from_UTC_Hours"]} hrs   {ppk_user_settings["EXIF_drift_correction_seconds"]:3d} sec' + \
                   f'   {ppk["nflash"]:6,d}   {ppk["Missing_Flashs"]:5d} {ppk["npix"]:6,d}     {ppk["exif_duptimes_count"]:5d}    {ppk_user_settings["User_marker"]:4.0f}' + \
        f'  {ppk_user_settings["Accuracy_Scale_Factor"]:4.3f} {ppk_user_settings["Accuracy_Offset_Meters"]:4.3f}m ' + \
        f' {ppk_user_settings["Method"].rjust(6)}      {ppk_user_settings["Output_Start_Time"]}  {ppk_user_settings["Output_End_Time"] }  '
  return hdr

junk='''
#@title Test: generate_output_header_2(ppk, ppk_user_settings,  odf )  {form-width: "20%"}
if __name__ == '__main__' and IN_COLAB:
  print(generate_output_header_2(ppk, ppk_user_settings ))
'''



In [ ]:
#@title def Generate Output Files()  { form-width: "25%", display-mode: "form" }
def generate_output_header(ppk, ppk_user_settings,  odf ):
  '''
  Generate the header for the output pix_pos file.

  Please use generate_output_header_2 instead.
  '''
  currentDT = datetime.datetime.now()
  print( f'# C. W. Wright https://github.com/lidar532  Generated: {str(currentDT)}',
        '\n# Generating Program:', ppk['Notebook Program'],'  Library Version:', ppk['ppk_flash_sync_version'],
        '\n# ',ppk_user_settings['User_Notes'],
        '\n#  Trajectory File:', ppk_user_settings['Trajectory_file_Name'],
        '\n#        EXIF File:', ppk_user_settings['Exif_File_Name'], "  ", " Photo_Count:", ppk['npix'],
        ' Applied Clock Drift Correction:', ppk_user_settings['EXIF_drift_correction_seconds'], 'Sec.  EXIF UTC Offset: ', ppk_user_settings['EXIF_Offset_from_UTC_Hours'],
        '\n# Flash Event File:', ppk_user_settings['Flash_Events_file_Name'], "  ", ppk['nflash'], ' Flash Events.  Base Station: ',
        ppk['GPS_base_station_name'], file=odf )


In [ ]:
#@title def generate_output_file( ppk, ppk_user_settings,  output_data )  { form-width: "25%", display-mode: "form" }
def generate_output_file( ppk, ppk_user_settings,  output_data ):
  # Generate the output file suitable for input to Agisoft Photoscan
  # Wrte the output data file
  if ppk['output_records'] == 0 :
    ppk['run'] == False
    display(HTML("<h1 style=""color:Red""><b> WARNING:  The photos do not overlap the Flash Events and no output file<br>"+
                 "can be generated. Make sure your trajectory is in UTC and not GPS Time,<br>"+
                 " and your exif_offset_seconds.</b></h1>"))
                 
    #print('*************************************************************************************************************')
    #print("* WARNING:  The photos do not overlap the Flash Events and no output file can be generated.                 *")
    #print("*            Make sure your trajectory is in UTC and not GPS Time, and your exif_offset_seconds is correct. *")
    #print('*************************************************************************************************************')
    return ''
  gen_output_file_name( ppk, ppk_user_settings)
  if ppk['debug'] > 0:
      print('generate_output_file()', ppk['ofn'])
      print('Computed output file name: ', ppk['ofn'] )
  odf = open(ppk['ofn'], 'w')

  generate_output_header(ppk, ppk_user_settings,  odf )
  output_data.to_csv( odf,
                  sep= '\t', index=False,
                  columns = [ 'pix','PIX_lat','PIX_lon','PIX_nad83h', 'Accuracy_Z', 'Flash_Ztime','raw_exif_ymd_hms', 
                      'GPS_nad83h','GPS_lat','GPS_lon','Q', 'ns', 'sdu', 'ratio', 'gps_sow', 'Seconds_Offset' ],
                  header = [
                      'Photos.DNG                         ', 
                      'PIX_Lat      ',
                      'PIX_Lon    ',
                      'PIX_Elev    ',
					  'Accuracy_Z    ',
                      'raw_exif_ymd_hms',
                      'Flash_Event_UTC_DATE_Time    ',
                      'GPS_Elevation',
                      'GPS_Lat',
                      'GPS_Lon',
                      'Q',
                      'N_Sats',
                      ' ZStdev    ',
                      ' Ratio',
					            'GPS_Time',
								'Seconds_Offset'
                  ],
                  float_format='%10.8f',
                 index_label = '# Date     UTC Time'
                )

  print( '# Footer ', file=odf)
  odf.close()
  print('Output File closed')
  return 'ok'



In [ ]:
#@title def extract_rtklib_settings( fn, ppk_user_settings ) { form-width: "25%", display-mode: "form" }
def extract_rtklib_settings( fn, ppk_user_settings ):
  # Sample input line
  # % obs start : 2019/02/25 16:49:39.5 UTC (week2042 146997.5s)
  traj_comments = OrderedDict()
  with open( ppk_user_settings['Trajectory_file_Name']) as ppk_traj_fdx:
      line = ppk_traj_fdx.readline()
      cnt = 1
      imp_file_cnt = 0
      while line:
          cnt = cnt + 1
          comment   = re.split("^%", line )                # Split out comments
          if len(comment) == 2:
              comment = re.sub("^ ","", comment[1] )       #
              comment = re.split(" : ", comment )
              if len(comment) == 2: 
                  var = comment[0]
                  var = var.rstrip()                       # Strip trailing spaces from var name
                  if var == 'inp file':                    # 
                      imp_file_cnt = imp_file_cnt + 1
                      var = var+"-"+str(imp_file_cnt)
                  val = comment[1].rstrip()                # strip trailing spaces and newline, Build list
                  traj_comments[var] = val
          if cnt >= 100: break
          line = ppk_traj_fdx.readline()
  return traj_comments

#@title def gen_horizontal_table(cshow, dlist, nspaces, caption="", width='50%'):
def gen_horizontal_table(cshow, dlist, nspaces, caption="", width='50%'):
  """
  Generates an HTML table string of dict values indexed by cshow.

  Inputs: 
      cshow:   list of indexes, 
      dlist:   dict_list, 
      nspaces: number of white spaces to add to the end
      caption: Set the caption of the table. Optional
      width:   Set the table width in % of the window

  Returns: an html table string
  """
  html_string = '<table width='+width+'<caption>'+caption+'<tr>'
  for x in cshow:
      html_string = html_string +'<th><center>'+x.ljust(nspaces)+'</center></th>'
      # print(x.ljust(nspaces), end='')
  html_string = html_string+'</tr><tr>'
  for x in cshow:
      html_string = html_string+'<td><center>'+dlist[x].ljust(nspaces)+'</center></td>'
      # print(traj_comments[x].ljust(nspaces), end='')
  html_string = html_string +'</tr></table>'
  return html_string

#@title def show_rtklib_settings( traj_comments )
def show_rtklib_settings( traj_comments ):
  """
  Display the values and settings extracted from the RTKlib position file as an HTML table.
  """
  width = '55%'     # Table width in % of screen
  cshow = (  'pos mode',  'freqs',  'solution',  'elev mask',  'dynamics',  'tidecorr' )
  s = gen_horizontal_table( cshow, traj_comments, 1, caption="RTKLib Settings", width=width  )

  cshow = (  'ionos opt', 'tropo opt',  'ephemeris',  'navi sys',  'amb res',  'val thres' )
  s = s+gen_horizontal_table( cshow, traj_comments, 1, width=width )

  cshow = ( 'program',   'antenna1' )
  s = s+gen_horizontal_table( cshow, traj_comments, 1, width=width )

  cshow = ( 'antenna2',  'ref pos'  )
  s = s+gen_horizontal_table( cshow, traj_comments, 1, width=width )

  cshow = ( 'inp file-1', 'inp file-2', 'inp file-3', 'inp file-4', 'inp file-5', 'inp file-6', 'inp file-7'  ) 
  s = s + "<table><caption>RTKlib GPS RINEX Rover, Base, Nav, Ephemris, Clock Files</caption><tr>\n"
  for a in cshow:
      if a in traj_comments: 
          s = s + "<th>"+a+"</th><td style=text-align:left;>"+traj_comments[a]+"</td></tr>"
  s = s + "</table>"
  display(HTML(s))
  return



In [ ]:
#@title def show_file_stats(ppk_user_settings, ppk_trj_df, exif_df, flash_stamps_df, output_df ) { form-width: "25%", display-mode: "form" }
def show_file_stats(ppk_user_settings, ppk_trj_df, exif_df, flash_stamps_df, output_df ):
  """
  Display stats on the input files, and a graphic showing the distribution of the times in in put files. 
  """
   # Disable decoding trajectory header for now.
  ### traj_comments = extract_rtklib_settings( ppk_user_settings['Trajectory_file_Name'] )
  ### show_rtklib_settings(  traj_comments )					
  first_photo_time = exif_df['Correct_exif_Ztime'].min()
  last_photo_time  = exif_df['Correct_exif_Ztime'].max()
  first_trj_time   = ppk_trj_df['Ztime'].min()
  last_trj_time    = ppk_trj_df['Ztime'].max()
  first_flash_time = flash_stamps_df['Flash_Ztime'].min()
  last_flash_time  = flash_stamps_df['Flash_Ztime'].max()
  out_min          = output_df['Flash_Ztime'].min()
  out_max          = output_df['Flash_Ztime'].max()
  
  print(f'last_flash_time: {last_flash_time}  last_photo_time: {last_photo_time}    ')
  tdiff_seconds = (last_flash_time - last_photo_time).total_seconds()
  estimated_offset_hours  = int(round(tdiff_seconds/3600))
  estimated_offset_seconds = round(tdiff_seconds - estimated_offset_hours*3600)
  
  # selecting components of datatime object
  # https://stackoverflow.com/questions/41782920/how-do-i-format-a-pandas-timedelta-object
  cds = exif_offset_seconds = ppk_user_settings['EXIF_drift_correction_seconds']
#  if exif_offset_seconds.components.days < 0 : 
#      cds = exif_offset_seconds.components.seconds - 60
#  else:
#      cds = exif_offset_seconds.components.seconds
  print("                             Executed:", datetime.datetime.now())
  print("Trajectory_GPS_to_UTC_Time_difference:", ppk_user_settings['Trajectory_GPS_to_UTC_Time_difference'] )
  #print("                            Platform:", platform.node(), " User:", os.getlogin(), " Version:", ppk_flash_sync_version) 
  print("                   Generating Program:", ppk['Notebook Program'])
  print("                           User Notes:", ppk_user_settings['User_Notes'])
  print("                       EXIF File Name:", ppk_user_settings['Exif_File_Name'] )
  print("                             PPK Traj:", ppk_user_settings['Trajectory_file_Name'] )
  print("                    Trajectory_Source:", ppk_user_settings['Trajectory_Source'] )
  print("                         Flash Events:", ppk_user_settings['Flash_Events_file_Name'] )
  print("                Output Directory File:", ppk_user_settings['PixPos_Directory'])
  print("           Output Photo Position File:", os.path.basename(ppk['ofn']))
  print("")
  print("              EXIF UTC Camera         ")
  print("Photo   Flash  Offset   Drift   Base")
  print("Count   Events Hours   Seconds Station      Start Time              End Time           Source")
  print("-------------------------------------------------------------------------------------------------------------")
  print(ppk['npix'], '',   
        ppk_user_settings['EXIF_Offset_from_UTC_Hours'],
        cds,
        ppk['GPS_base_station_name'],
        format_single_time(first_photo_time), format_single_time(last_photo_time), "Camera EXIF",
        sep="\t")

  print( '',ppk['nflash'],"\t\t", format_single_time(first_flash_time), format_single_time(last_flash_time), "Flash Events", sep="\t" )
  print( "\t\t\t\t", format_single_time(first_trj_time), format_single_time(last_trj_time), "PPK", sep="\t" )
  if ppk['output_records'] > 0 :
    print( ppk['output_records'],"\t\t\t", format_single_time(out_min), format_single_time(out_max), "Output", sep="\t" )
  else :
    print( '\t \t\t\t\t** No EXIF Photo date/times that matche Flash Events **')
  if ( estimated_offset_hours ):
    display(HTML("<h1 style=background-color:red;color:white;><b>Estimated required EXIF_Offset_from_UTC_Hours: "+str(estimated_offset_hours)+" Hours</b></h1>"))
  if ( estimated_offset_seconds ):
    display(HTML("<h1 style=background-color:red;color:white;><b>Estimated required EXIF_drift_correction_seconds: "+str(estimated_offset_seconds)+" Seconds</b></h1>")) 
  print("------------------------------------------------------------------------------------------------------------")
  return



In [ ]:
#@title def gen_q_colors(output_data) { form-width: "25%", display-mode: "form" }
def gen_q_colors(output_data):
  '''    
  Generate an array of colors based on the "Q" of each ppk photo position.

  Note:
  -----
  'Q' values range from 1-5, with nan for non existent values. Values mapped as follows:

  1   Green
  2   Orange
  4   Blue
  5   Red
  nan Black

  Parameters: 
  -----------
    output_data 

  Returns:
  --------
   An array of colors names.
  '''
  q = output_data['Q']
  q_colors = np.where( q == 1.0,    'LightGreen',   q) 
  q_colors = np.where( q == 2.0,    'Orange', q_colors)
  q_colors = np.where( q == 4.0,    'Blue',   q_colors)
  q_colors = np.where( q == 5.0,    'Red',    q_colors)
  q_colors = np.where( np.isnan(q), 'Black',  q_colors)
  return q_colors



In [ ]:
#@title def gen_q_sizes(output_data): { form-width: "25%", display-mode: "form" }
def gen_q_sizes(output_data):
  # adjust the point size based on "Q".  Make Q2, Q3 values larger
  q = output_data['Q']
  q_size = np.where( q == 1,  2.5, q )
  q_size = np.where( q_size == 2, 5, q_size) 
  q_size = np.where( q_size == 4, 5, q_size) 
  q_size = np.where( q_size == 5, 5, q_size)
  return q_size



In [ ]:
#@title def photo_event_time_distro(ppk, ppk_user_settings,  exif_df, flash_stamps_df ): { form-width: "25%", display-mode: "form" }
# plot the distribution of photo event times around each one second mark
def photo_event_time_distro(ppk, ppk_user_settings,  exif_df, flash_stamps_df ):
  # Compute the time difference between the flash time stamps, and the Ricoh clock EXIF values
  #flash_stamps_df.head()
  if ppk['output_records'] == 0:
    return
  if ppk['debug'] > 0:
      print("photo_event_time_distro()")
      print( ppk['nflash'], " Flash Time Stamps found in File: ", ppk_user_settings['Flash_Events_file_Name'])
      print(  ppk['npix'], "      Photos flound in EXIF file: ", ppk_user_settings['Exif_File_Name'], "\n")
  # print("", flash_stamps_df['Ztime'] - exif_df['Ztime'])
  tdifs = (flash_stamps_df['Flash_Ztime'] - exif_df['Correct_exif_Ztime'])  / np.timedelta64(1, 's')
  tdifs = tdifs.where( (tdifs < 9.0) & ( tdifs > -9.0) )

#   %matplotlib inline

  plt.rcParams["figure.dpi"] = 100    
  plt.xlabel('Photo Event time offset from each GPS Second')
  plt.ylabel('Photo Events')
  thist = tdifs.plot( bins=1000,
                          x = 'Seconds',
                          kind='hist',
                          title=f'Photo Event Time Distribution. {tdifs.count()} total events.',
                          figsize = (10,3),
                          range=[-0.1,0.1]
                         )
  plt.show()
  return


junk='''
if __name__ == '__main__':
  photo_event_time_distro( ppk,  ppk_user_settings, exif_selected_df, flashs_selected_df )
'''


In [ ]:
#@title def plot_times( exif_df, flash_stamps_df, ppk_trj_df, output_df ): { form-width: "25%", display-mode: "form" }
def plot_times( exif_df, flash_stamps_df, ppk_trj_df, output_df ):
  """
  Plots the flash event times, the corrected exif times, and the traj times. using matplotlib. Useful
  to check time alignment of the three input files.

  """
#   %matplotlib inline
  title = 'Date / Times within each File'
  x_label = 'Date / Time'
  y_label = 'Event'
  plt.rcParams["figure.dpi"] = 100
  plt.figure(figsize=(15,1) )
  x = pd.to_datetime(exif_df['Correct_exif_Ztime'])
  q_color = gen_q_colors( output_df)
  q_size  = gen_q_sizes(output_df)

  
  x = ppk_trj_df['Ztime']
  y = np.empty( len(x))
  y.fill(1)
  plt.plot_date( x, y,
      markersize = 15.0, color="green", label="GPS Trajectory Times"
  )

  x = exif_df['Correct_exif_Ztime']
  y = np.empty( len(x))
  y.fill(1.0)
  plt.plot_date( x,y,
      markersize = 10.0, color="Black", marker="s", label="Photo Times"
  )
    
  x = flash_stamps_df['Flash_Ztime']
  y = np.empty( len(x))
  y.fill(1.0)
  plt.plot_date( x,y,
      markersize = 4.0, color="Orange", marker="s", label="Flash Event Times"      
  )
  
  plt.xticks(rotation = 35)
  plt.title(title)
  plt.xlabel(x_label)
  #    plt.ylabel(y_label)
  plt.legend(loc=(0.0, 1.0))
  plt.show()
  return



In [ ]:
#@title def plot_pix_elevations( ppk, ppk_user_settings,  output_data ): { form-width: "25%", display-mode: "form" }
def plot_pix_elevations( ppk, ppk_user_settings,  output_data ):
  """
  Plots the elevations vs sow (Seconds of the week) using matplotlib.

  """
  if ppk['output_records'] == 0:
    return
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Enter plot_pix_elevations()')
  title = 'PPK Photo Elevations (meters)'
  x_label = 'Seconds of the Week'
  y_label = 'Camera PPK Elevation (meters)'
  plt.rcParams["figure.dpi"] = 100
  q_color = gen_q_colors( output_data )
  q_size  = gen_q_sizes(  output_data )
  output_data.plot.scatter( 
      x='sow', y='PIX_nad83h', 
      s=q_size, 
      c=q_color, 
      figsize=(10,3)
  )
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Leave plot_pix_elevations()')
  return



In [ ]:
#@title def plot_planview_orig( ppk, output_data, proj='meters' ): { form-width: "25%", display-mode: "form" }
def plot_planview_orig( ppk, output_data, proj='meters' ):
  """Plots a plan view of the camera locations using matplotlib.  
  It can plot in UTM, meters, or LatLon degrees.  It uses the computed values
  found in the output_data global variable. The plotted points are colored
  according to the PPK Quality factor. Q=1 is Green, Q=2 is Mustard, Q=5 is Red.
  The quality flag Q and the marker color means: 

  1: Fixed, 2: Float, 4: DGPS, 5: Single

     Parameters
     ----------
     proj = ['latlon', 'utm', 'meters']

     Returns
     -------
     Nothing

     References: http://www.rtklib.com/prog/manual_2.2.2.pdf
  """

  if ppk['output_records'] == 0:
    return
#   %matplotlib inline
  lat = output_data['PIX_lat'].values
  lon = output_data['PIX_lon'].values

  # generate an array of colors based on the "Q" of each ppk photo position
  q = output_data['Q']
  q_colors = np.where(        q == 1 , 'green', q )
  q_colors = np.where(q_colors == '2', 'orange', q_colors )
  q_colors = np.where(q_colors == '4', 'blue',   q_colors )
  q_colors = np.where(q_colors == '5', 'red',    q_colors )

  # adjust the point size based on "Q".  Make Q2, Q3 values larger
  q_size = np.where( q == 1,      2.5, q )
  q_size = np.where( q_size == 2, 5, q_size) 
  q_size = np.where( q_size == 4, 5, q_size) 
  q_size = np.where( q_size == 5, 5, q_size)

  if proj == 'meters' or proj == 'utm':        
      # Compute the UTM zone
      z = utm_zone( lon[0])
      proj_string = '+proj=utm +zone={z:1.0f}, +north +ellps=WGS84 +datum=NAD83 +units=m +no_defs'.format(z=z)
      print(proj_string)
      myProj = Proj( proj_string)
      x, y = myProj( lon, lat)
      if proj == 'meters':
          x = x - int(x.min())
          y = y - int(y.min())
          title = 'PPK Camera Positions in Meters'
          x_label = 'Meters'
          y_label = 'Meters'

      elif proj == 'utm':
          title = 'PPK Camera Positions in UTM Meters'
          x_label = 'UTM Meters Easting'
          y_label = 'UTM Meters Northing'

  elif proj == 'latlon':
      title = 'PPK Camera Positions in Lat / Lon'
      x_label = 'Longitude (Degrees)'
      y_label = 'Latitude (degrees)'
      x = lon
      y = lat

  plt.figure(figsize=(10, 8), dpi=100)
  plt.grid(True, linestyle=':')
  plt.scatter( x, y, s=q_size, c=q_colors )
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()
  return



In [ ]:
#@title Testing: Test plot_planview() { form-width: "30%", display-mode: "form" }
# Testing code below for plot_planview()
debug_plot_planview = False
if __name__ == '__main__' and debug_plot_planview:
  mp = "ESRI Imagery" #@param ["ESRI Imagery", "Open Street Maps", "WorldMap" ]
  point_scale = 3 #@param [1, 1.5, 2.1, 2.5, 3, 5, 7.5, 10] {type:"raw"}
  plot_planview(ppk, selected_output, mp=mp,  point_scale=point_scale, legend='PPK')


In [ ]:
#@title def utm_zone( lon ): { form-width: "25%", display-mode: "form" }
#@markdown  Compute the UTM zone from a single longitude value
def utm_zone( lon ):
  """utm_zone( lon )
  Given the longitude as input, returns the UTM zone number.
  Parameters:
  --------
  Input:   lon   Single longitude value.
  Returns: UTM Zone
  """
  z = int((np.floor((lon + 180.0) / 6 ) % 60) + 1)
  return z



In [ ]:
#@title def plot_pix_stdevs( ppk, ppk_user_settings, output_data ): { form-width: "25%", display-mode: "form" }
def plot_pix_stdevs( ppk, ppk_user_settings, output_data ):
  """
  Plots (matplotlib) separate Std Devs for Z, Northing, and Easting vs sow (Seconds of the week)
  using matplotlib. The points are colored by the GPS Quality factor.
  Inputs: ppk                The ppk data structure
          ppk_user_settings  The ppk user settings
          output_data        The output_data structure to be plotted.
  Output: A scatter plot of time vs the X,Y and Z StdDevs.  Plot is colorized by GPS Quality factor.  
  """
  if ppk['output_records'] == 0:
    return
  
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Enter plot_pix_stdevs()')
  height = 2
  title = 'PPK Photo Elevations (meters)'
  x_label = 'Seconds of the Week'
  y_label = 'Camera PPK Elevation (meters)'
  plt.rcParams["figure.dpi"] = 100
  q_color = gen_q_colors( output_data )
  q_size  = gen_q_sizes( output_data )

  title = 'Z PPK Std Dev per Photo '
  x_label = 'Seconds of the Week'
  y_label = 'Z Stdev (meters)'

  x = output_data['sow']
  plt.figure(figsize=(10, height), dpi=100)
  plt.grid(True, linestyle=':')
  plt.scatter( x, output_data['sdu'], s=1, color='blue')
  plt.scatter( x, output_data['sdn'], s=1, color='red')
  plt.scatter( x, output_data['sde'], s=1, color='green')
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Leave plot_pix_stdevs()')
  return




In [ ]:
#@title def process_cww_ppk_files(ppk, ppk_user_settings, plots=True, Debug=0, Gen_output = True ): { form-width: "25%", display-mode: "form" }
def process_cww_ppk_files(ppk, ppk_user_settings, plots=True, Debug=0, Gen_output = True ):
# This is the main functions for users to call to process a dataset.
  """Process PPK position file, Camera EXIF file, and Photo Time Stamp file and produce an
  output file readable by Agisoft Photoscan.  

  Parameters:
  -----------
  plots = [ True | False ]
  Debug = [ 0, 1, 2]                  0= has no debug output, 1=some debug output, 2=max debug output
  Gen_output = [ True | False ]   True = Generate output file, False causes no output file to be written.
  """
  global output_df, exif_df, ppk_trj_df, flash_stamps_df, run, xdebug, output_file_name
  ppk['run'] = True
  ofn                            =     check_input_files( ppk, ppk_user_settings )
  if ppk['run']: exif_df         =             load_exif( ppk, ppk_user_settings )
  if ppk['run'] and ppk_user_settings['Trajectory_Source'] == 'RTKlib': ppk_trj_df      =   load_rtklib_trj( ppk, ppk_user_settings )
  if ppk['run'] and ppk_user_settings['Trajectory_Source'] == 'PPP-Ca': ppk_trj_df      =   load_ppkca_trj( ppk, ppk_user_settings )  
  if ppk['run']: flash_stamps_df 														=   load_k706_flash_events( ppk, ppk_user_settings )
  if ppk['run']: output_df       														=   interpolate_flash_positions( ppk, ppk_user_settings, exif_df, ppk_trj_df, flash_stamps_df ) 

  if ppk['run'] and ppk_user_settings['Generate_Output_File']=='Yes': 
    err = generate_output_file( ppk, ppk_user_settings, output_df )
  if ppk['run'] and ppk_user_settings['Generate_Output_File'] == 'No':
    output_file_name = "Ouput File is turned off. NO OUTPUT FILE GENERATED"
    display(HTML("<h1><b>"+output_file_name+"</b></h1>"))
    
  if ppk['run'] and ppk_user_settings['Show_File_Stats']: show_file_stats(ppk_user_settings, ppk_trj_df, exif_df, flash_stamps_df, output_df )
  if ppk['run']:
    if ppk_user_settings['Plot_Times']:                    plot_times( exif_df, flash_stamps_df, ppk_trj_df,  output_df  )
    if ppk_user_settings['Show_Flash_event_Distribution']: photo_event_time_distro( ppk,  ppk_user_settings, exif_df, flash_stamps_df )
    if ppk_user_settings['Show_Photo_Location_Plan_View']: plot_planview(       ppk, output_df )
    if ppk_user_settings['Show_Photo_Elevations']:         plot_pix_elevations( ppk, ppk_user_settings,  output_df )
    if ppk_user_settings['Show_XYZ_Std_Devs']:             plot_pix_stdevs(     ppk, ppk_user_settings,  output_df )
  if ppk['run'] == False: 
    print("Process aborted, No output file was generated.") 
    print(f'error message: {ppk["run_error"]}')
  return


def is_main_module():
  return __name__ == '__main__' and '__file__' not in globals()



In [ ]:
#@title def plot_planview( ppk, output_data, mp="ESRI Imagery", point_scale=1.0, proj='meters', legend='' ): { form-width: "25%", display-mode: "form" }
import pandas as pd
from pathlib import Path

if not Path('/usr/local/bin/pyproj').is_file():
  !pip install pyproj

import bokeh.io
bokeh.io.output_notebook()
import bokeh.plotting
from bokeh.plotting import figure
from bokeh.plotting import ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
from pyproj import Proj, transform, CRS


def plot_planview( ppk, output_data, mp="ESRI Imagery", point_scale=1.0, proj='meters', legend='' ):
  '''
  Plots the gps position on a user selectable background map.  Uses Bokeh.

  Parameters:
  -----------
  ppk : dict.       Not used within, but here for legacy.
  output_data : pdf Pandas data fram with the data to plot, and info columns.
  mp="ESRI Imagery" default, others are: "WikiMedia", "WorldMap", "Open Street Maps", "STAMEN_TERRAIN"
  legend=''  Label to apply.
  point_scale=1, value to multiply the point size by.
  proj='utm'    unused. legacy.

  Returns
    Nothing.
  '''
  point_scale = float(point_scale)
  TOOLS = "pan,wheel_zoom,box_zoom,reset, save, undo, redo, hover"
  TOOLTIPS = [
      ("UTC:", "@zhms"),
      ("Pix:", "@pix"),
      ("Shutter:", "@shutter"),
      ("ISO:", "@iso"),
      #("index", "$index"),  #,
      #("(x,y)", "($x, $y)"),
      ("Lat:", "@PIX_lat{0.00000000}"),
      ("Lon:", "@PIX_lon{0.00000000}"),
      ("Elev:", "{@PIX_nad83h} m"),
      ("SDU", "@sdu{0.000} m"),
      ("Q:", "@Q"),
      ("NSat:", "@ns")
  ]
  #screenProj = Proj(init='epsg:3857')     # 
  #wgs84Proj = Proj(init='epsg:4326')    # WGS-84

  screenProj = CRS('EPSG:3857')
  wgs84Proj  = CRS('epsg:4326')

  map_border = (output_data['PIX_lat'].max() - output_data['PIX_lat'].min()) * .3
  lat_min = output_data['PIX_lat'].min() - map_border
  lat_max = output_data['PIX_lat'].max() + map_border
  lon_min = output_data['PIX_lon'].min() - map_border
  lon_max = output_data['PIX_lon'].max() + map_border

    # generate an array of colors based on the GPS "Q" of each ppk photo position
  q = output_data['Q']
  q_colors = np.where(        q == 1 , 'lightgreen', q )
  q_colors = np.where(q_colors == '2', 'orange', q_colors )
  q_colors = np.where(q_colors == '4', 'blue',   q_colors )
  q_colors = np.where(q_colors == '5', 'red',    q_colors )
  q_colors = np.where( np.isnan(q), 'Black',     q_colors )
  q_colors = gen_q_colors( output_data )

  # adjust the point size based on GPS "Q".  Make Q2, Q3 values larger
  q_size = np.where( q == 1,     2.5*point_scale, q )
  q_size = np.where( q_size == 2, 10*point_scale, q_size) 
  q_size = np.where( q_size == 4, 40*point_scale, q_size) 
  q_size = np.where( q_size == 5, 60*point_scale, q_size)
  q_size = gen_q_sizes( output_data )

  #world_lon1, world_lat1 = transform(wgs84Proj,screenProj,lon_min,lat_max)
  #world_lon2, world_lat2 = transform(wgs84Proj,screenProj,lon_max,lat_min)
  #gps_lats, gps_lons = transform(wgs84Proj, screenProj, output_data['PIX_lon'], output_data['PIX_lat'])

  from pyproj import Transformer
  transformer = Transformer.from_crs( wgs84Proj, screenProj)

  world_lon1, world_lat1 = transformer.transform(lat_min, lon_max)
  world_lon2, world_lat2 = transformer.transform(lat_max, lon_min)
  gps_lons, gps_lats = transformer.transform(output_data['PIX_lat'], output_data['PIX_lon'])

    #cartodb = get_provider(CARTODBPOSITRON)
  if mp == "ESRI Imagery":
    cartodb = get_provider(ESRI_IMAGERY)
  elif mp == "WikiMedia":
    cartodb = get_provider(WIKIMEDIA)
  elif mp == "WorldMap":
    cartodb = get_provider(CARTODBPOSITRON)
  elif mp == "Open Street Maps":
    cartodb = get_provider(OSM)
  elif mp == "STAMEN_TERRAIN":
    cartodb = STAMEN_TERRAIN

  map_df = output_data.copy()
  map_df['x'] = gps_lons
  map_df['y'] = gps_lats
  map_df['q_color'] = q_colors
  map_df['q_size'] = q_size

  fig = figure(plot_width=800, plot_height=800,
              tooltips=TOOLTIPS, tools=TOOLS,
              x_range=(world_lon1, world_lon2),
              y_range=(world_lat1, world_lat2),
              x_axis_type="mercator", y_axis_type="mercator",
              output_backend="webgl"
              )
  pix_names = ColumnDataSource( output_data)
  #fig.circle(y=gps_lons, x=gps_lats, legend_label="", color=q_colors, size=q_size )
  fig.circle(y='y', x='x', legend_label=legend, color='q_color', size='q_size', source=map_df)

  fig.add_tile(cartodb)
  show(fig)



In [ ]:
#@title Defs Bokeh defs.  open_new_plot(), open_time_plot() Timeline plots.{ form-width: "35%", display-mode: "form" }

import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.events import ButtonClick
from bokeh.models import Button
from bokeh.models import Button, CustomJS
from bokeh.plotting import ColumnDataSource
from bokeh.models.tools import HoverTool
#Output_Start_Time = '2020-08-02 15:52:09' #@param {type:"string"}
#Output_End_Time   = '2020-08-02 15:52:20' #@param {type:"string"}

# del hover_tips

# See: https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#formatting-tooltip-fields
#HoverTool( 
#    tooltips=[("Index", "$index"),
#              ("x", "$x")
#              ]
#)

'''
  TOOLTIPS = [
      ("UTC:", "@zhms"),
      ("Pix:", "@pix"),
      ("Shutter:", "@shutter"),
      ("ISO:", "@iso"),
      #("index", "$index"),  #,
      #("(x,y)", "($x, $y)"),
      ("Lat:", "@PIX_lat{0.00000000}"),
      ("Lon:", "@PIX_lon{0.00000000}"),
      ("Elev:", "{@PIX_nad83h} m"),
      ("SDU", "@sdu{0.000} m"),
      ("Q:", "@Q"),
      ("NSat:", "@ns")
  ]

# 'wobble_ms'

hover_tips = [
            ("Time at Cursor:", '$x{%F %T.%3N}'),
            ("Corrected EXIF", "@Correct_exif_Ztime{%F %T.%3N}"),
            ("Raw EXIF:", "@exif_time{%F %T.%3N}"),
            ("Index", "$index"),
            ("Pix:", "@pix"),
            ("GPS Ztime:","@Ztime{%F %T.%3N}"),
            ("GPS lat:", "@GPS_lat{0.00000000}"),
            ("GPS lon:", "@GPS_lon{0.00000000}"),
            ("GPS_Alt:", "@GPS_nad83h{0.000}")
            ]
'''

#hover_tool.formatters = { "@{Correct_exif_Ztime}": "datetime"}

def open_new_plot(width=1000, height=500, 
                tools='pan,wheel_zoom,box_zoom,reset,undo, redo, save, hover'
                ):
  '''
  '''
  try:
    settings
  except:
    NameError
    settings = {}
    settings['plot_width'] = width
    settings['plot_height'] = height
    WebGL = "Enabled"

  if WebGL == 'Enabled':
    p0 = figure( plot_width = settings['plot_width'], #tooltips=hover_tips,
                plot_height= settings['plot_height'],
                output_backend = "webgl",
                      tools=tools
              )
  else:
      p0 = figure( plot_width = settings['plot_width'], #tooltips=hover_tips,
                plot_height= settings['plot_height'],
                      tools=tools
              )
      
  print(f'Tools: {tools}')
  p0.toolbar.autohide                 = True
  p0.title.text_font_size             = '18pt'
  p0.yaxis.axis_label_text_font_size  = '16pt'
  p0.yaxis.major_label_text_font_size = '14pt'
  p0.xaxis.major_label_text_font_size = '14pt'
  p0.xaxis.axis_label_text_font_size  = '16pt'
  p0.legend.click_policy = "hide"
  #p0.add_tools(tips)
  p0.hover.formatters={"$x":"datetime", 
                      "@Correct_exif_Ztime": "datetime", 
                      "@exif_time": "datetime",
                      "@Ztime": "datetime",
                      }
  return p0


def open_time_plot(width=1000, height=500, 
                  tools='pan,xwheel_zoom,box_zoom,reset,undo, redo, my_hover_tips'
                  ):
  '''
  '''
  p0 = open_new_plot(width, height, tools=tools)
  p0.yaxis.axis_label                 = "Elevation (meters)"
  p0.xaxis.axis_label                 = "Time"
  p0.xaxis.major_label_orientation    = 1.
  p0.xaxis.formatter = bkm.DatetimeTickFormatter(hours=  ['%H:%M:%S'], 
                                                  hourmin=['%H:%M:%S'],
                                                  minutes=['%H:%M:%S'],
                                                  minsec= ['%H:%M:%S'], 
                                                  seconds=['%H:%M:%S']
                                                )
  return p0



In [ ]:
#@title def def time_line( trj=[], events=[],  exif=[], exif_dups=[], markers=[] ): Camera Sync Check EXIF, Flash-Events, & Trajectory{ form-width: "35%", display-mode: "form" }

def time_line( trj=[], events=[],  exif=[], exif_dups=[], markers=[] ):
  '''
  Generate an interactive timeline.
  '''
  output_notebook()
  pt1 = open_time_plot(1500, 520, 
                      tools='xpan, xwheel_zoom, box_zoom, reset, redo, undo, save')
  # 'xpan, xwheel_zoom, box_zoom, reset, redo, undo, save, crosshair'
  pt1.toolbar.active_inspect = None
  pt1.toolbar.active_scroll = 'auto'
  pt1.toolbar.active_drag  = 'auto'
  pt1.y_range.end    = 0.1;
  pt1.y_range.start = -.15;
  pt1.title.text =   'EXIF Corrections: Seconds:'+\
  str(ppk_user_settings['EXIF_drift_correction_seconds'])+\
  ', UTC Hours:'+str(ppk_user_settings['EXIF_Offset_from_UTC_Hours'])
  pt1.yaxis.axis_label                 = "Seconds (Blue)"
  pt1.xaxis.axis_label                 = "Time ( UTC )"
  pt1.yaxis.bounds=[-1,.1]

  # Trajectory
  if len(trj) == 0:
    print('You need to load a trajectory file')
  else:
    ppk_trj_df_hover_source = ColumnDataSource(data = trj )
    p1 = pt1.rect(                          # Plot Trj df
          #pd.to_datetime(ppk_trj_df['Ztime']),
          x = 'Ztime',
          y =-0.075,
          width=110.0, height=0.015,
          color='green',
          legend_label='GNSS Trajectory',
          source = ppk_trj_df_hover_source
          )
    # https://stackoverflow.com/questions/49282078/multiple-hovertools-for-different-lines-bokeh
    pt1.add_tools(HoverTool( description='Trajectory',
        renderers = [p1], tooltips=[
            (     "Date:", '$x{%F}'),           
            (  "GPS UTC:","@Ztime{%T.%3N}"),
            (" GPS Time:", "@gps_sow{%T.%3N}"),
            ( "GPS lat:", "@GPS_lat{0.00000000}"),
            ( "GPS lon:", "@GPS_lon{0.00000000}"),
            ( "GPS_Alt:", "@GPS_nad83h{0.000} m"),
            (       "Q:", "@Q{0}"),
            (      "NS:", "@ns{0}"),
            ( "Stdev Z:", "@sdu{0.000} m")
            #(   "Cursor:", '$x{%T.%3N}'),
            #(    "Index:", "$index")            
           ],
           formatters={'$x':'datetime', '@Ztime':'datetime', '@gps_sow':'datetime'}
    ))

  #=================================================================================
  # EXIF Duplicate Photos
  #=================================================================================
  if len(exif_dups) == 0:
    print('You need to load the EXIF duplicates.')
  else:
    exif_dups_df_hover_source = ColumnDataSource(data = exif_dups[ First_Pix_Index:] )
    p2 = pt1.diamond(                          # Plot dups corrected EXIF times
        #pd.to_datetime(exif_duptimes['Correct_exif_Ztime']),
        x = 'Correct_exif_Ztime',
        y=-0.12, 
        color='red',
        legend_label='Duplicate EXIF Time',
        size=30.0,
        source = exif_dups_df_hover_source
        )
    pt1.add_tools(HoverTool( description='Dup. TS. Photos',
    renderers = [p2], tooltips=[
        (          "Date:", '$x{%F}'),        
        ("Corrected EXIF:", "@Correct_exif_Ztime{%T.%3N}"),
        ("Raw EXIF:", "@exif_time{%F %T.%3N}"),
        ("Pix:", "@pix")
        #(        "Cursor:", '$x{%F %T.%3N}'),
        #("Index", "$index")        
        ],
        formatters={'$x':'datetime', '@Correct_exif_Ztime':'datetime', '@exif_time':'datetime'}
))

  #=================================================================================
  # EXIF Photos
  #=================================================================================
  if len(exif) == 0:
    print('You need to load an EXIF file first.')
  else:
    exif_df_hover_source = ColumnDataSource(data = exif[ First_Pix_Index:] )
    p3 = pt1.diamond(                          # Plot corrected EXIF times
        #pd.to_datetime(exif_df[ First_Pix_Index:]['Correct_exif_Ztime']),
        x = 'Correct_exif_Ztime',
        y = -.1, 
        color='black',
        legend_label='EXIF (Z)',
        source = exif_df_hover_source,
        size=30.0
        )
    pt1.add_tools(HoverTool( description='Photos',
        renderers = [p3], tooltips=[
            #(        "Cursor:", '$x{%F %T.%3N}'),
            #("Index", "$index"),                                     
            (          "Date:", '$x{%F}'),            
            ("Corrected EXIF:", "@Correct_exif_Ztime{%T.%3N}"),
            ("Raw EXIF:", "@exif_time{%F %T.%3N}"),
            ("Pix:", "@pix")           
           ],
           formatters={'$x':'datetime', '@Correct_exif_Ztime':'datetime', '@exif_time':'datetime'}
    ))

  #=================================================================================
  # Flash Events
  #================================================================================= 
  if len(events) == 0:
    print('You need to load the FLASH Stamps file first.')
  else:
    flash_stamps_df['wobble_ms'] = flash_stamps_df['Flash_Ztime'].dt.microsecond / 1e6
    wobble = flash_stamps_df['wobble_ms']
    wobble = np.where( wobble > 0.5, wobble-1.0, wobble)
    flash_stamps_df['wobble_ms'] = wobble 
    flash_stamps_hover_source = ColumnDataSource(data = events )
    p4 = pt1.diamond_cross(                          # Plot the Flash Times
        #pd.to_datetime(flash_stamps_df['Flash_Ztime']),
        x = 'Flash_Ztime',
        y = -0.11,
        color='orange',
        legend_label='Flash Events (Z)',
        size=10.0,
        source = flash_stamps_hover_source
        )
    pt1.add_tools(HoverTool( description='Flash Events',
      renderers = [p4], tooltips=[                              
          (          "Date:", '$x{%F}'),
          ("Flash Event UTC:", "@Flash_Ztime{%F %T.%3N}"),
          ("Wobble:", "@wobble_ms{0.000} Sec")
          ],
          formatters={'$x':'datetime', '@Flash_Ztime':'datetime' }
    ))
  
    #=================================================================================
    # Event time Wobble
    #=================================================================================
    pt1.diamond_cross(                          # Plot the Flash Times
        pd.to_datetime(events['Flash_Ztime']),
        flash_stamps_df['wobble_ms'],
        color='blue',
        legend_label='ms offset',
        size=5.0
        )
  
  #=================================================================================
  # User Time Markers
  #=================================================================================
  # See this for named colors: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
  for marker in markers:
    pt1.rect(pd.to_datetime(marker['time']), -0.02, 40.0, .2, color=marker['color'])
   
  pt1.legend.click_policy = "hide"
  show(pt1)
  return pt1

debug_time_line = False;
if __name__ == '__main__' and debug_time_line:
  markers = [ 
             {'time':Output_Start_Time, 'color':'Lime'},
             {'time':Output_End_Time, 'color':'Red'}
            ]
  ptx = time_line( trj      = ppk_trj_df,
                  exif      = exif_df, 
                  exif_dups = exif_duptimes,
                  events    = flash_stamps_df,
                  markers   = markers
                  )


In [ ]:
#@title End of defs. { form-width: "30%", display-mode: "form" }
if __name__ == '__main__':
  print(f'{datetime.datetime.utcnow()}: End of defs loaded in main:  {cwwppkgeotaglib_asof} ' )

# PPK-2-PixPos ( Colab GUI )

In [ ]:
#@title Test: ppk-2-pixpos settings.. {form-width: "30%"}
def libs_loaded():
  rv = 'yes'
  try:
    cwwppkgeotaglib_asof
  except:
    rv = ''
  return rv


if __name__ == '__main__' and IN_COLAB and libs_loaded() == 'yes':
  #@markdown **Settings:**
  #@markdown ----
  #@markdown **Example Output_Start_Time:**   2020-05-04 15:44:00
  Trajectory_Source      = "RTKlib" #@param ["RTKlib", "PPP-Ca"]
  Output_Start_Time = '2021/10/13 17:00:00' #@param {type:"string"}
  Output_End_Time   = '2021/10/13 23:59:59' #@param {type:"string"}
  EXIF_drift_correction_seconds =  0#@param {type:"integer" }
  EXIF_Offset_from_UTC_Hours =  0#@param {type:"integer"}
  User_marker = 3300 #@param ["1100", "2200", "3300", "4400", "5500", "6600", "7700", "8800", "9900"] {type:"raw"}
  First_Pix_Index =  0#@param {type:"integer"} 
  User_Notes = '3DR Solo.  ' #@param {type:"string"}
  Base_station_ID = 'P310' #@param {type:"string"}

  #@markdown ----
  #@markdown #Files and Folders:
  PixPos_Directory =  "/content/Gdrive/Missions/2021-1013-Ca-Josh_Logan/2021-1013-Ca-Josh_Logan_GNSS/aircraft/pix_pos" #@param {type:"string"}
  Flash_Events_file_Name = "/content/Gdrive/Missions/2021-1013-Ca-Josh_Logan/2021-1013-Ca-Josh_Logan_GNSS/aircraft/raw/2021-1013-all-events.txt" #@param {type:"string"}
  Trajectory_file_Name   = "/content/Gdrive/Missions/2021-1013-Ca-Josh_Logan/2021-1013-Ca-Josh_Logan_GNSS/aircraft/trajectories/2021-1013-all-aircraft-P310-RK15N-cmb-pos.txt" #@param {type:"string"}
  exif_fn = "/content/Gdrive/Missions/2021-1013-Ca-Josh_Logan/2021-1013-Ca-Josh_Logan_GNSS/aircraft/exif/2021-10-13_JenkinsonLake_all_img_exif.txt" #@param {type:"string"}  

  #@markdown ---
  #@markdown #SfM Accuracy:
  #@markdown SfM_Accuracy = (Z Standard Deviation) * Accuracy_Scale_Factor +  Accuracy_Offset_Meters
  Accuracy_Scale_Factor = 2.0 #@param {type:"number"}
  Accuracy_Offset_Meters = 0.1 #@param {type:"number"}

  #@markdown ---
  #@markdown #Stats, Graphs & Plots:

  #Plot_Times = True #@param {type:"boolean" }
  #Show_File_Stats = True #@param {type:"boolean" }
  #Show_Flash_event_Distribution = True #@param {type:"boolean" 
  #Show_Photo_Location_Plan_View = True #@param {type:"boolean"}
  #Show_Photo_Elevations = True #@param { type: "boolean" }
  
  #@markdown ---
  #@markdown #Map settings:
  Map_Type = "ESRI Imagery" #@param ["ESRI Imagery", "Open Street Maps", "WorldMap" ]
  Point_Scale = 1.5 #@param [1, 1.5, 2.1, 2.5, 3, 5, 7.5, 10] {type:"raw"}
  #@markdown ---

  output_column_list = [
            'pix','PIX_lat','PIX_lon','PIX_nad83h','Accuracy_Z','sdu','Q','ns',
            'shutter','iso','Flash_Ztime','gps_sow','date','zhms','sow'
            ]

  # get the name of this notebook.
  from requests import get
  my_file_filename = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']
  

  ppk['Notebook Program'] = my_file_filename
  ppk_user_settings['PixPos_Directory']       = PixPos_Directory
  ppk_user_settings['Trajectory_Source']      = Trajectory_Source
  ppk_user_settings['Flash_Events_file_Name'] = Flash_Events_file_Name
  ppk_user_settings['Accuracy_Scale_Factor']  = Accuracy_Scale_Factor
  ppk_user_settings['Accuracy_Offset_Meters'] = Accuracy_Offset_Meters
  ppk_user_settings['Trajectory_file_Name']   = Trajectory_file_Name
  ppk_user_settings['Exif_File_Name']         = exif_fn
  ppk_user_settings['EXIF_drift_correction_seconds'] = EXIF_drift_correction_seconds 
  ppk_user_settings['EXIF_Offset_from_UTC_Hours'] = EXIF_Offset_from_UTC_Hours
  ppk_user_settings['User_marker']            = User_marker
  ppk_user_settings['Output_Start_Time']      = Output_Start_Time
  ppk_user_settings['Output_End_Time']        = Output_End_Time
  ppk_user_settings['User_Notes']             = User_Notes
  ppk_user_settings['First_Pix_Index']        = First_Pix_Index
  ppk_user_settings['Base_station_ID']        = Base_station_ID

  #ppk_user_settings['Plot_Times']                       = Plot_Times
  #ppk_user_settings['Show_File_Stats']                  = Show_File_Stats
  #ppk_user_settings['Show_Flash_event_Distribution']    = Show_Flash_event_Distribution
  #ppk_user_settings['Show_Photo_Location_Plan_View']    = Show_Photo_Location_Plan_View
  #ppk_user_settings['Show_Photo_Elevations']            = Show_Photo_Elevations



In [ ]:
#@title Test: ppk-2-pixpos run.. {form-width: "30%"}
  #@markdown # Output Settings:
if __name__ == '__main__' and IN_COLAB:
  if libs_loaded() != 'yes':
    print('************************************************')
    print('*  You need to load all the above defs first.  *')
    print('************************************************')
  else:
    Sync_Using_Corrected_EXIF_Time = 'Yes' #@param ["Yes", "No"]
    Generate_Output_File           = "No" #@param ["Yes", "No"]
    #Force_Fit = 'No' #@param ["Yes", "No"]


    if Sync_Using_Corrected_EXIF_Time == 'Yes':
      ppk_user_settings['Method']     = 'Time'
    else:
          ppk_user_settings['Method'] = 'Index'

    #====================================================================================================
    # Load the camera flash events, the GNSS trajectory, and the photo/camera data
    # flash_stamps_df   is the Camera Flash Event papdas dataframe holding the MEP (Mid Exposure Pulse)
    #                   event time.
    # ppk_trj           is the PPK trajectory pandas dataframe containing the trajectory data generated
    #                   by either RTKlib or Grafnav. 
    # exif_df           is the pandas dataframe containing the photo file name and various information
    #                   extracted from the photo EXIF metadata.
    #====================================================================================================

    # Get the flash events
    flash_stamps_df = load_k706_flash_events( ppk, ppk_user_settings, duplicates='remove' )

    # Examine the trajectory file to try and determine if generated by RTKlib or Grafnav.
    trj_gen         = determine_trajectory_generator( ppk_user_settings['Trajectory_file_Name'] )

    # Now load the trajectory to ppk_trj using the appropriate loader.
    if trj_gen == 'rtklib':
      ppk_trj = load_rtklib_trj( ppk, ppk_user_settings )
    elif trj_gen == 'grafnav':
      ppk_trj = load_grafnav_trj( ppk, ppk_user_settings )
    elif trj_gen == 'none':
      print(f'No valid Trajectory file given.')
    ppk['Trajectory_Generator'] = trj_gen
    ppk_trj_df = ppk_trj

    # Load the Photo EXIF file which contains the photo file names and EXIF values.
    exif_df = load_exif( ppk, ppk_user_settings, duplicates='remove', show_duplicates=False )

    # Now trim the ppk, flash_stamps, and the exif data frames to the Output_Start_Time & Output_End_Time
    ppk_trj_selected_df  = ppk_trj_df.loc[Output_Start_Time:Output_End_Time]
    flashs_selected_df   = flash_stamps_df.loc[Output_Start_Time:Output_End_Time]

    # Trim the exif to the Output_Start_Time & Output_End_Time, and also reduce the
    # data to pix, shutter, iso, and Correct_exif_Ztime.
    exif_selected_df     = exif_df[['pix','shutter','iso', 'Correct_exif_Ztime']].loc[Output_Start_Time:Output_End_Time]

    # Add an 'Idx' column
    flashs_selected_df.insert(0, 'Idx', list(range(len(flashs_selected_df.index))) )

    # Interpolate lat,lon, and elevation values from ppk to the flash_selected_interp_df dataframe.
    flash_selected_interp_df = interpolate_flash_positions_2( ppk, ppk_user_settings, ppk_trj, flashs_selected_df )

    if Sync_Using_Corrected_EXIF_Time == 'Yes':   # EXIF time Sync method
      odfn = gen_output_file_name( ppk, ppk_user_settings,  tail='-EXIFTS-pixpos.txt')
      ppk['ofn'] = odfn
      selected_output = sync_with_exif(ppk, ppk_user_settings, exif_selected_df, flash_selected_interp_df )
      ppk['Missing_Flashs'] = exif_selected_df["pix"].count() - selected_output["pix"].count()
      print(f'Output_data records generated: {selected_output["pix"].count()}.  EXIF Photos: {exif_selected_df["pix"].count()}.' )
      print('')
      selected_output['Accuracy_Z'] = selected_output['Accuracy_Z'] + (User_marker * 0.00000001)
      Generate_output_2(odfn, selected_output, Generate_output=Generate_Output_File )

      if Generate_Output_File == 'Yes':
        print(f'\nPix-Pos Output written to: {odfn}')
      else:
        print('\nNo Pix-Pos output file was generated.')
    else: # Sequencial Sync method
      odfn = gen_output_file_name( ppk, ppk_user_settings,  tail='-SEQSYNC-pixpos.txt')

      Accuracy_Scale_Factor =  ppk_user_settings['Accuracy_Scale_Factor']            = Accuracy_Scale_Factor
      Accuracy_Offset_Meters = ppk_user_settings['Accuracy_Offset_Meters']           = Accuracy_Offset_Meters

      #def generate_ppk_pix( Output_Start_Time, Output_End_Time, exif_df, flash_stamps_df, ppk_trj_df, odfn ):
      
      # List of columns to be taken from flash_stamps_df and added to the output records
      flash_list = ['PIX_lat', 'PIX_lon', 'PIX_nad83h','sow', 'Flash_Ztime', 'Zulu Offset', 'gps_sow']

      # List of columns, in desired order, to be sent to the output file.
      out_list  = ['pix','PIX_lat', 'PIX_lon', 'PIX_nad83h', 'Accuracy', 'Q', 'ns','shutter', 'iso', 'sow', 'Flash_Ztime', 'GPS_Time' ]
      #out_list  = ['pix','PIX_lat', 'PIX_lon', 'PIX_nad83h', 'Accuracy', 'Q', 'ns','shutter', 'iso', 'sow', 'Flash_Ztime']

      # 1. Slice out all the flash events between the start and end times
      # 2. Add a simple integer index column to use for the join.  We do not want to use the time based index.
      flashs_selected_df   = flash_stamps_df.loc[Output_Start_Time:Output_End_Time]
      flash_selected_interp_df = interpolate_flash_positions_2( ppk, ppk_user_settings, ppk_trj, flashs_selected_df )
      flashs_to_use = flash_selected_interp_df[flash_list].loc[Output_Start_Time:Output_End_Time]
      exif_out      = exif_df[['pix','shutter','iso']].loc[    Output_Start_Time:Output_End_Time]
      # 1. Extract useful info from the GPS trajectory file and join into the flashs_to_use df
      flashs_to_use = flashs_to_use.join( ppk_trj_df, how='left')

      # Compute and add in a column for GPS Time.  This column helps Grafnav users overlay their results
      flashs_to_use['GPS_Time'] = flashs_to_use['Flash_Ztime'] - pd.to_timedelta(flashs_to_use['Zulu Offset'], unit='s')

      # Compute Accuracy and create a column for it. **Must** be done before switching to simple count index
      # Then fill missing GPS data, Q, sdu, etc. with padded values.
      flashs_to_use['Accuracy'] = flashs_to_use['sdu'] * Accuracy_Scale_Factor + Accuracy_Offset_Meters
      flashs_to_use = flashs_to_use.fillna(method='pad')

      # Add in a simple integer index column
      flashs_to_use['Idx'] = list(range(len(flashs_to_use.index)))
      flashs_to_use = flashs_to_use.set_index('Idx')

      # 1. Slice out all of the photos from the exif_df that are between start and end times
      # 2. Add a simple integer index column.  We want to avoid using the time index mostly
      # due to issues with the Sony cameras jumping a second or two when changing batteries.
      exif_out['Idx'] = list(range(len(exif_out.index)))
      exif_out        = exif_out.set_index('Idx')

      fc = flashs_to_use.count()[0]
      ec = exif_out.count()[0]

      #print(f'Before User Selected Force Fit . fc: {fc} ec: {ec}')
      #if fc > ec:
      #  flashs_to_use = flashs_to_use[0:ec]
      #elif ec > fc:
      #  exif_out = exif_out[0:fc]
      #print(f' After User Selected Force Fit . fc: {fc} ec: {ec}')

      if flashs_to_use.count()[0] == exif_out.count()[0]:
        selected_output = exif_out.iloc[:].join(flashs_to_use).iloc[:]
        selected_output['Accuracy_Z'] = selected_output['sdu']*ppk_user_settings['Accuracy_Scale_Factor']+ppk_user_settings['Accuracy_Offset_Meters']
        selected_output['Accuracy_Z'] = selected_output['Accuracy_Z'] + (User_marker * 0.00000001)
        if Generate_Output_File == 'Yes':
          print(f'\nPix-Pos Output written to: {odfn}')
          with open(odfn, 'w') as odf:
            generate_output_header(ppk, ppk_user_settings,  odf )
            Generate_output_2(odfn, selected_output, Generate_output=Generate_Output_File )
            '''
            print( selected_output[out_list].to_string( formatters= {'pix':'{:s}'.format,
                                                                  'PIX_lat':'{:12.9f}'.format,
                                                                  'PIX_lon':'{:12.9f}'.format,
                                                                  'PIX_nad83h':'{:8.3f}'.format,
                                                                  'Accuracy':'{:4.2f}'.format,
                                                                  'Q':'{:1.0f}'.format,
                                                                  'ns':'{:2.0f}'.format,
                                                                  'sow':'{:15.8f}'.format}, index=False), file=odf )
                                                                  '''
          print(f'{flashs_to_use.count()[0]} photo positions written to: {odfn}')
        else:
          print('\nNo Pix-Pos output file was generated.')
      else:
        print(f'The number of pix ({exif_out.count()[0]}) does not equal the number of flash\n\
        events ({flashs_to_use.count()[0]}). The number of photos must match the number of flash events. Adjust\n\
        Output_Start_Time and/or Output_End_Time to reject photos or flash events that do not have\n\
        an associated photo or flash event. You can also adjust the EXIF_drift_correction_seconds to\n\
        bring the photos in alignment with the flash events.\
        ')


In [ ]:
#@title **Examine your Results** { form-width: "30%", display-mode: "form" }
if __name__ == '__main__':
  import ipywidgets as ipw
  from IPython.display import clear_output 


  sel_stats = ipw.ToggleButton(description="File Stats")
  sel_event_distro = ipw.ToggleButton(description="Event Distro Plot")
  sel_map_view = ipw.ToggleButton(description="Map View Plot")
  sel_pix_elev = ipw.ToggleButton(description="Photo Elevation Plot")
  sel_time_line = ipw.ToggleButton(description="Event Time Line")
  button = ipw.Button(
      description='Generate Selected',
      disabled=False,
      button_style='', # 'success', 'info', 'warning', 'danger' or ''
      tooltip='Click to generate the selected products.',
      icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  def vars_exist(lst):
    '''
    Return False if any of the  variables in lst are not defined.

    Parameters:
    lst:  A list of names of variables that must exist.

    Returns:
    list:  ('name', True|False) Where 'name' is the name of the variable, and True if it exists, False if not.
    '''
    for df in lst:
      if df not in globals():
        return (df,False)
    return ('',True)


  click_count = 0;
  def button_clicked(v):
    global click_count
    click_count = click_count + 1
    #print(f'clicked: click_count={click_count}  v={v}')
    clear_output()
    show_menu()
    rv = vars_exist(('ppk_user_settings', 'ppk_trj_df', 'exif_df', 'flash_stamps_df',   'selected_output'))
    if not rv[1]:
      print(f'You need to load {rv[0]} first.')
    else:
      if sel_stats.value:
        show_file_stats(ppk_user_settings, ppk_trj_df, exif_df, flash_stamps_df,   selected_output )
      if sel_event_distro.value:
        photo_event_time_distro( ppk,  ppk_user_settings, exif_selected_df, flashs_selected_df )
      if sel_map_view.value:
        plot_planview(       ppk, selected_output,  legend=odfn, mp=Map_Type, point_scale=Point_Scale  )
      if sel_pix_elev.value:
        plot_pix_elevations( ppk, ppk_user_settings,  selected_output )
      if sel_time_line.value:
        markers = [ 
                {'time':Output_Start_Time, 'color':'Lime'},
                {'time':Output_End_Time, 'color':'Red'}
              ]
        print(markers)
        ptx = time_line( trj      = ppk_trj_df,
                        exif      = exif_df, 
                        exif_dups = exif_duptimes,
                        events    = flash_stamps_df,
                        markers   = markers
                        )


  def show_menu():
    display(
        ipw.Label(value="Click to select the products to display."),
        ipw.HBox([sel_time_line, sel_stats, sel_event_distro, sel_map_view, sel_pix_elev]),
        button
        )
  button.on_click( button_clicked )

  show_menu()


# Testing and dev Code

In [ ]:
#@title **Testing:** Display ppk_trj_df, flash_stamps_df, exif_selected_df, exif_duptimes, exif_duptimes { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  display(      ppk_trj_df[0:3])
  display(flash_stamps_df[0:3])
  display(exif_selected_df[0:3])
  display(exif_duptimes, exif_duptimes[0:3])

In [ ]:
#@title Testing: Set Start and Stop times
#@markdown **Example Output_Start_Time:**   2020-05-04 15:44:00
if __name__ == '__main__' and IN_COLAB:
  Output_Start_Time = '' #@param {type:"string"}
  Output_End_Time   = '' #@param {type:"string"}



## IPW widget GUI dev cells

In [ ]:
#@title File name input widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  import ipywidgets as ipw
  #=======================================================================
  # File GUI widgets
  # see: https://github.com/jupyter-widgets/ipywidgets/blob/8.0.0a6/docs/source/examples/Widget%20Styling.ipynb
  # for more on layout and config.
  #=======================================================================
  fw = {'width':'60%', 'description_width':'100px'}
  w_pix_pos_dir  = ipw.Text(description='PixPos Dir:',      layout=fw, style=fw, 
                            description_tooltips='Copy and paste the PixPos directory name where you want things saved.')
  w_flash_events = ipw.Text(description='Flash Events:',    layout=fw,  style=fw, )
  w_trj_file     = ipw.Text(description='Trajectory:',      layout=fw, style=fw,  )
  w_exif         = ipw.Text(description='EXIF:',            layout=fw, style=fw, )
  w_files = ipw.VBox( [ 
      ipw.HTML(value="<h2>Files & Paths:</h2>"),
      w_pix_pos_dir,
      w_flash_events,
      w_trj_file,
      w_exif               
  ] )

display(w_files)

In [ ]:
#@title Settings widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  #=======================================================================
  # Settings GUI widgets
  #=======================================================================
  w_set_trj_src = ipw.Dropdown(
      #value='RTKLib',
      #placeholder='RTKLib',
      options=['RTKLib', 'Grafnav', 'CA PPK', 'PosPac'],
      description='Trajectory Src:',
      #value = 'RTKLib',
      layout={'width':'200px'},
      ensure_option=True,
      disabled=False
  )

  w_set_user_mark = ipw.Dropdown(
      # value='John',
      placeholder='1100',
      options=['1100', '2200', '3300', '4400', '5500'],
      description='Marker:',
      layout={'width':'150px'},
      #value = '1100',
      ensure_option=True,
      disabled=False
  )

  w_set_base_id = ipw.Text(description='GNSS Base:', layout={'width':'200px'})
  w_trj_mark_base = ipw.HBox( [w_set_trj_src, w_set_user_mark, w_set_base_id])
  display(w_trj_mark_base)


In [ ]:
#@title Map plot Settings widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  w_set_map_type = ipw.Dropdown(
      # value='John',
      placeholder='ESRI Imagery',
      options=["ESRI Imagery", "Open Street Maps", "WorldMap" ],
      description='Map Type',
      #value = 'ESRI Imagery',
      layout={'width':'250px'},
      ensure_option=True,
      disabled=False
  )

  w_map_point_scale = ipw.Dropdown(
      # value='John',
      placeholder='1',
      options=['1', '1.5', '2.1', '2.5', '3', '5', '7.5', '10'],
      description='Map Point Size',
      ensure_option=True,
      layout={'width':'150px'},
      disabled=False
  )

  w_set_map = ipw.HBox([w_set_map_type, w_map_point_scale])

  display(w_set_map)




In [ ]:
#@title Start, End Date & Time widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  w_set_sd = ipw.DatePicker(
      description='Start Date',
      layout={'width':'250px'},
      disabled=False
  )

  w_set_st   = ipw.Text(layout={'width':'80px'}, placeholder='HHMMSS')
  w_set_sdt  = ipw.HBox([w_set_sd, w_set_st ])

  w_set_ed = ipw.DatePicker(
      description='End Date',
      layout={'width':'250px'},
      disabled=False
  )
  w_set_et  = ipw.Text(layout={'width':'80px'},  placeholder='HHMMSS')
  w_set_edt  = ipw.HBox([w_set_ed, w_set_et ])

  w_set_exif_sec_offset = bounded_int_text = ipw.BoundedIntText(step=1, description='Sec Offset:',   layout={'width':'150px'}, min=-86400, max=86400)
  w_set_exif_hr_offset  = bounded_int_text = ipw.BoundedIntText(step=1, description='Hours Offset:', layout={'width':'150px'}, min=-1000,  max=1000)
  w_set_exif_day_offset = bounded_int_text = ipw.BoundedIntText(step=1, description='Days Offset:', layout={'width':'150px'}, min=-365,  max=365)
  w_set_first_pix_idx = ipw.BoundedIntText(step=1, description='Pix to Skip:', layout={'width':'150px'}, min=0,  max=10000)
  w_set_exif_offsets = ipw.HBox([w_set_exif_day_offset, w_set_exif_hr_offset, w_set_exif_sec_offset])

  w_set_start_end = ipw.HBox( [ipw.VBox(
      [
       ipw.HTML(value='<h2>Output Start & Stop Date / Times</h2>'),
       w_set_sdt, 
       w_set_edt]
       ),
       w_set_first_pix_idx,
       w_set_exif_offsets
    ]
  )




  display(w_set_start_end )



In [ ]:
#@title User Notes widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':  
  w_set_user_notes = ipw.Text(description='User Note:', layout={'width':'60%'})

  display(w_set_user_notes)

In [ ]:
#@title SFM accuracy settings widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  w_set_sfm_accuracy_scale = ipw.BoundedFloatText(step=0.01, description='SFM Scale:',  layout={'width':'150px'}, min=0.0, max=10.0)
  w_set_sfm_accuracy_bias  = ipw.BoundedFloatText(step=0.01, description='SFM Bias:',   layout={'width':'150px'}, min=0.0, max=10.0)
  w_set_sfm_accuracy = ipw.HBox([w_set_sfm_accuracy_scale, w_set_sfm_accuracy_bias])

  display(w_set_sfm_accuracy)

In [ ]:
#@title Display GUI widgets { form-width: "25%", display-mode: "form" }
if __name__ == '__main__':
  display(  
          w_set_start_end,          
          ipw.HBox( [w_set_map_type, w_map_point_scale, w_set_first_pix_idx]), 
          ipw.HBox([ipw.VBox( [w_set_sdt, w_set_edt]), w_set_exif_offsets ]),
          w_set_sfm_accuracy,
          w_set_user_notes,
          w_files 
          )

## Run the following cell to get sample data for testing from github.

In [ ]:
#@title Get same data from github. { form-width: "25%", display-mode: "form" }
# Execute only when running as __main__.  Requires linux (colabs only for now)
if __name__ == '__main__':
  ! rm -rf /content/sample_data/
  ! rm -rf /content/PPK_Sample_data/
  ! git clone https://github.com/lidar532/PPK_Sample_data.git
  print('Operation completed.')

# Old depreciated code below.

In [ ]:
#@title Old original user interface cell for pix-2-pos.
if __name__ == '__main__' and IN_COLAB:
  print(f"{ppk['ppk_flash_sync_version']}   IN_COLAB:{IN_COLAB}")
  #@title PPK Geo Tagging for the CWW-PPK-v2 GNSS System
  #@markdown #Input Data Files
  Trajectory_Source      = "RTKlib" #@param ["RTKlib", "PPP-Ca"]
  Trajectory_GPS_to_UTC_Time_difference =   0#@param {type:"integer" }
  Exif_File_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/exif/2019-0311-Solo-ckscript-F4-exif.csv" #@param {type:"string"}
  Trajectory_file_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/gps/GP184856-NG15P-FLD7.pos" #@param {type:"string"}
  Flash_Events_file_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/gps/GP184856.TXT" #@param {type:"string"}
  #@markdown ---
  #@markdown #Camera EXIF Time Adjustments
  Photos_to_Skip =  0#@param {type:"integer" }
  EXIF_drift_correction_seconds =  -37#@param {type:"integer" }
  EXIF_Offset_from_UTC_Hours =   4#@param {type:"integer"}
  #@markdown ---
  #@markdown #SfM Accuracy
  #@markdown SfM_Accuracy = (Z Standard Deviation) * Accuracy_Scale_Factor +  Accuracy_Offset_Meters
  Accuracy_Scale_Factor = 2.0 #@param {type:"number"}
  Accuracy_Offset_Meters = 0.05 #@param {type:"number"}

  #@markdown ---
  #@markdown #Output File
  PixPos_Directory = "/content/" #@param {type:"string"}
  Base_station_ID = "test" #@param {type:"string"}
  User_Notes = "Test" #@param {type:"string"}
  Generate_Output_File = "No" #@param ["No", "Yes"]

  #@markdown ---
  #@markdown #Stats, Graphs & Plots
  Plot_Times = True #@param {type:"boolean" }
  Show_File_Stats = True #@param {type:"boolean" }
  Show_Flash_event_Distribution = True #@param {type:"boolean" }
  Show_XYZ_Std_Devs = False #@param {type:"boolean"} 
  Show_Photo_Location_Plan_View = False #@param {type:"boolean"}
  Show_Photo_Elevations = False #@param { type: "boolean" }
  #@markdown ---
  Debugging_Output = "None" #@param ["None", "Function EntryExit", ""]
  #@markdown ---

# Run this cell to process the data.
if __name__ == '__main__':
  ppk['Notebook Program'] = f'cwwppkgeotaglib as of {asof}'
  print('\nCWW PPK Geotag Library version: ', ppk['ppk_flash_sync_version'])
  ppk_user_settings['Photos_to_Skip']                   = Photos_to_Skip
  ppk_user_settings['Accuracy_Scale_Factor']            = Accuracy_Scale_Factor
  ppk_user_settings['Accuracy_Offset_Meters']           = Accuracy_Offset_Meters
  ppk_user_settings['PixPos_Directory']                 = PixPos_Directory
  ppk_user_settings['Trajectory_Source']                = Trajectory_Source
  ppk_user_settings['Exif_File_Name']                   = '/' +  Exif_File_Name
  ppk_user_settings['Flash_Events_file_Name']           = '/' + Flash_Events_file_Name
  ppk_user_settings['Trajectory_file_Name']             = '/' + Trajectory_file_Name
  ppk_user_settings['EXIF_drift_correction_seconds']    = EXIF_drift_correction_seconds
  ppk_user_settings['EXIF_Offset_from_UTC_Hours']       = EXIF_Offset_from_UTC_Hours
  ppk_user_settings['Generate_Output_File']             = Generate_Output_File
  ppk_user_settings['Base_station_ID']                  = Base_station_ID
  ppk_user_settings['User_Notes']                       = User_Notes
  ppk_user_settings['Generate_Output_File']             = Generate_Output_File
  ppk_user_settings['Plot_Times']                       = Plot_Times
  ppk_user_settings['Show_File_Stats']                  = Show_File_Stats
  ppk_user_settings['Show_Flash_event_Distribution']    = Show_Flash_event_Distribution
  ppk_user_settings['Show_XYZ_Std_Devs']                = Show_XYZ_Std_Devs
  ppk_user_settings['Show_Photo_Location_Plan_View']    = Show_Photo_Location_Plan_View
  ppk_user_settings['Show_Photo_Elevations']            = Show_Photo_Elevations
  ppk_user_settings['Debugging_Output']                 = Debugging_Output
  ppk_user_settings['Generate_Output_File']             = Generate_Output_File

  process_cww_ppk_files(ppk, ppk_user_settings  )
  if ppk['run'] == False:
    print(f"\n{ppk['run_error']}\n")
